In [1]:
#Import
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
#from biasmodels import MagnificationBias, GalaxyBias, EvolutionBias
from CAMBsolver import Solver
from multipole_signal import Signal
import fishermat as fm
import pickle as pk

#Set current directory
import os
import sys

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd())))
sys.path.append(SCRIPT_DIR)
print(SCRIPT_DIR)

#Configuration
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large,
          'text.usetex': True}
plt.rcParams.update(params)


/Users/danielsb/Documents/GitHub


In [2]:
# Redshift bins

z_bins = np.array([0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])

# Separations

dist = [
    np.arange(20, 164, 4),
    np.arange(32, 164, 4),
]

params_dict0 = {
    'h' : 0.677,
    'As' : 2.05e-9,
    'ns' : 0.965,
    'Ob' : 0.0490, # This is here the density parameter of Baryons
    'Om' : 0.3111, # This is here the density parameter of Cold Dark Matter
}

steps = [1e-3, 1e-1, 1e-4, 1e-2, 1e-2]

# Check symmetricity

In [3]:
# Load the Covariance Matrices

CovMatrixSKA_50x50 = np.array(fm.cov_matrix_survey(d = dist[0], z_bins=z_bins, pixel_size=4, small_off_diagonal=True, rtol=1e-4, split=[50,50]))
CovMatrixSKA_30x70 = np.array(fm.cov_matrix_survey(d = dist[0], z_bins=z_bins, pixel_size=4, small_off_diagonal=True, rtol=1e-4, split=[30,70]))
CovMatrixSKA_70x30 = np.array(fm.cov_matrix_survey(d = dist[0], z_bins=z_bins, pixel_size=4, small_off_diagonal=True, rtol=1e-4, split=[70,30]))


 ... Loading Covariance Matrices [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 


 ... Success! ... 


 ... Loading Covariance Matrices [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 


 ... Success! ... 


 ... Loading Covariance Matrices [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 


 ... Success! ... 



In [4]:
np.linalg.det(CovMatrixSKA_50x50), np.linalg.det(CovMatrixSKA_30x70), np.linalg.det(CovMatrixSKA_70x30)

(array([-0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,  0.,  0., -0.,
        -0.,  0.,  0.,  0., -0., -0.]),
 array([-0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
        -0., -0., -0.,  0., -0., -0.]),
 array([ 0., -0., -0., -0., -0., -0., -0., -0.,  0., -0., -0.,  0.,  0.,
        -0., -0., -0.,  0.,  0., -0.]))

In [5]:
# Check if Cov Matrices are symmetric:

for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_50x50[n], CovMatrixSKA_50x50[n].T, rtol = 1e-20), 
          np.allclose(CovMatrixSKA_30x70[n], CovMatrixSKA_30x70[n].T, rtol = 1e-20),
          np.allclose(CovMatrixSKA_70x30[n], CovMatrixSKA_70x30[n].T, rtol = 1e-20))


0 True True True
1 True True True
2 True True True
3 True True True
4 True True True
5 True True True
6 True True True
7 True True True
8 True True True
9 True True True
10 True True True
11 True True True
12 True True True
13 True True True
14 True True True
15 True True True
16 True True True
17 True True True
18 True True True


In [6]:
# Setting small off-diagonal values to zero

CovMatrixSKA_50x50_samll_offdiag_to_zero = fm.cov_matrix_survey(d = dist[0], z_bins=z_bins, pixel_size=4, small_off_diagonal=False, rtol=1e-5, split=[50,50])
CovMatrixSKA_30x70_samll_offdiag_to_zero = fm.cov_matrix_survey(d = dist[0], z_bins=z_bins, pixel_size=4, small_off_diagonal=False, rtol=1e-5, split=[30,70])


 ... Loading Covariance Matrices [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 


 ... Removing small off-diagonal terms vs diagonal terms ... 


 Relative difference = 1e-05 


 ... Success! ... 


 ... Loading Covariance Matrices [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 


 ... Removing small off-diagonal terms vs diagonal terms ... 


 Relative difference = 1e-05 


 ... Success! ... 



In [7]:
# Check if Cov Matrices are symmetric:

for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_50x50_samll_offdiag_to_zero[n], CovMatrixSKA_50x50_samll_offdiag_to_zero[n].T, rtol = 1e-20), np.allclose(CovMatrixSKA_30x70_samll_offdiag_to_zero[n], CovMatrixSKA_30x70_samll_offdiag_to_zero[n].T, rtol = 1e-20))

0 True True
1 True True
2 True True
3 True True
4 True True
5 True True
6 True True
7 True True
8 True True
9 True True
10 True True
11 True True
12 True True
13 True True
14 True True
15 True True
16 True True
17 True True
18 True True


In [42]:
# Load the INVERSE covariance matrices

Inv_CovMatrixSKA_50x50 = fm.inverse_cov_matrix_survey(d = dist[0], z_bins=z_bins, pixel_size=4, small_off_diagonal=True, rtol=1e-4, split=[50,50])
Inv_CovMatrixSKA_30x70 = fm.inverse_cov_matrix_survey(d = dist[0], z_bins=z_bins, pixel_size=4, small_off_diagonal=True, rtol=1e-4, split=[30,70])
Inv_CovMatrixSKA_70x30 = fm.inverse_cov_matrix_survey(d = dist[0], z_bins=z_bins, pixel_size=4, small_off_diagonal=True, rtol=1e-4, split=[70,30])


 ... Loading Covariance Matrices [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 


 ... Computing the Inverse [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 


 ... Loading Covariance Matrices [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 


 ... Computing the Inverse [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 


 ... Loading Covariance Matrices [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 


 ... Computing the Inverse [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 



In [9]:
# Check if Inverse Cov Matrices are symmetric:

for n, z in enumerate(z_bins):
    print(n, np.allclose(Inv_CovMatrixSKA_50x50[n], Inv_CovMatrixSKA_50x50[n].T, rtol = 1e-4), np.allclose(Inv_CovMatrixSKA_30x70[n], Inv_CovMatrixSKA_30x70[n].T, rtol = 1e-4))

0 True True
1 True True
2 True True
3 True True
4 True True
5 True True
6 True True
7 True True
8 True True
9 True True
10 True True
11 True True
12 True True
13 True True
14 True True
15 True True
16 True True
17 True True
18 True True


In [10]:
Inv_CovMatrixSKA_50x50_hand = [np.linalg.inv(CovMatrixSKA_50x50[i]) for i,_ in enumerate(z_bins)]

for n, z in enumerate(z_bins):
    print(n, np.allclose(Inv_CovMatrixSKA_50x50[n], Inv_CovMatrixSKA_50x50_hand[n], rtol = 1e-20), n, np.allclose(Inv_CovMatrixSKA_50x50[n], Inv_CovMatrixSKA_50x50_hand[n].T, rtol = 1e-3))

0 True 0 True
1 True 1 True
2 True 2 True
3 True 3 True
4 True 4 True
5 True 5 True
6 True 6 True
7 True 7 True
8 True 8 True
9 True 9 True
10 True 10 True
11 True 11 True
12 True 12 True
13 True 13 True
14 True 14 True
15 True 15 True
16 True 16 True
17 True 17 True
18 True 18 True


In [11]:
Higher_Prec_CovMatrixSKA_50x50 = [CovMatrixSKA_50x50[i].astype(np.float64) for i,_ in enumerate(z_bins)]

In [12]:
Inv_Higher_Prec_CovMatrixSKA_50x50 = [np.linalg.inv(Higher_Prec_CovMatrixSKA_50x50[i]) for i,_ in enumerate(z_bins)]

In [13]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(Inv_Higher_Prec_CovMatrixSKA_50x50[n], Inv_Higher_Prec_CovMatrixSKA_50x50[n].T, rtol = 1e-3))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


# Joint Covariance Matrix 50x30 [CC ONLY]

## Check Cauchy-Schwarz inequality  

The Cauchy-Schwarz inequality implies:

$$ \mathrm{COV}(X,Y) \leq \sqrt{\mathrm{VAR}(X)\cdot\mathrm{VAR}(Y)}$$

In [14]:
CovMatrixSKA_joint = fm.cov_matrix_joint(d=dist[0], z_bins=z_bins, small_off_diagonal=True, rtol=1e-4, splits=[50,30], contributions='CC')


 ... Loading Covariance Matrices (Joint-splittings Analysis) [CC] [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 



In [15]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol = 1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [16]:
np.shape(CovMatrixSKA_50x50), np.shape(CovMatrixSKA_30x70), np.shape(CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 648, 648))

In [17]:
CovMatrixSKA_50x30 = np.array([CovMatrixSKA_joint[k][:324, 324:] for k,_ in enumerate(z_bins)])
CovMatrixSKA_30x50 = np.array([CovMatrixSKA_joint[k][324:, :324] for k,_ in enumerate(z_bins)])

In [18]:
np.shape(CovMatrixSKA_50x30), np.shape(CovMatrixSKA_30x50)

((19, 324, 324), (19, 324, 324))

In [19]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_50x30[n], CovMatrixSKA_30x50[n].T, rtol=1e-10), np.allclose(CovMatrixSKA_30x50[n], CovMatrixSKA_50x30[n].T, rtol = 1e-10))

0 True True
1 True True
2 True True
3 True True
4 True True
5 True True
6 True True
7 True True
8 True True
9 True True
10 True True
11 True True
12 True True
13 True True
14 True True
15 True True
16 True True
17 True True
18 True True


In [20]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol=1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [21]:
np.shape(CovMatrixSKA_50x30), np.shape(CovMatrixSKA_30x50)

((19, 324, 324), (19, 324, 324))

In [22]:
CovMatrixSKA_50x30[0,0,0], CovMatrixSKA_30x50[0,0,0]

(1.759172859468612e-05, 1.759172859468612e-05)

In [23]:
CovMatrixSKA_50x50[0,0,0], CovMatrixSKA_30x70[0,0,0]

(1.1406613424594502e-05, 2.728761334220845e-05)

In [24]:
CovMatrixSKA_50x30[0,17,267], CovMatrixSKA_30x50[0,267,17]

(-4.190495416728882e-09, -4.190495416728882e-09)

In [25]:
CovMatrixSKA_50x50[0,17,267], CovMatrixSKA_30x70[0,267,17]

(-4.188698962835081e-09, -5.405294849542528e-09)

## Checking directly from Joint Cov Matrix 50x30

Compare off diagonal elements (covariances) versus the correspondent diagonal ones (variances).

In [26]:
np.shape(CovMatrixSKA_joint), type(CovMatrixSKA_joint)

((19, 648, 648), list)

In [27]:
36*9

324

In [28]:
print(' Monopole BB (d=d), 50x50 = ', CovMatrixSKA_joint[0][0,0],'\n','Monopole BB (d=d), 30x70 = ', CovMatrixSKA_joint[0][324,324] ) 

 Monopole BB (d=d), 50x50 =  1.1406613424594502e-05 
 Monopole BB (d=d), 30x70 =  2.728761334220845e-05


In [29]:
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][0,324],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][324,0] ) 

 Monopole BB (d=d), 50x30 =  1.759172859468612e-05 
 Monopole BB (d=d), 30x50 =  1.759172859468612e-05


In [30]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][0,324], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][324,0])

0.15 True True
0.25 True True
0.35 True True
0.45 True True
0.55 True True
0.65 True True
0.75 True True
0.85 True True
0.95 True True
1.05 True True
1.15 True True
1.25 True True
1.35 True True
1.45 True True
1.55 True True
1.65 True True
1.75 True True
1.85 True True
1.95 True True


In [31]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]), CovMatrixSKA_joint[k][0,324])

0.15 1.764254110592856e-05 1.759172859468612e-05
0.25 7.014350303522421e-06 6.98731530466759e-06
0.35 3.904079159778656e-06 3.882676937714368e-06
0.45 2.5784765980908824e-06 2.5574319017803916e-06
0.55 1.8931986511136867e-06 1.869507799160479e-06
0.65 1.4969289169443364e-06 1.4681771688573775e-06
0.75 1.2532721988093958e-06 1.216178869326269e-06
0.85 1.1007803772214066e-06 1.0506751825877635e-06
0.95 1.0104386161984314e-06 9.39593695752212e-07
1.05 9.703620669081107e-07 8.652478347707248e-07
1.15 9.797925594098938e-07 8.173506398847174e-07
1.25 1.0534917202039827e-06 7.897336639125897e-07
1.35 1.2326007500359773e-06 7.78689068985697e-07
1.45 1.6096843827924573e-06 7.820860328394859e-07
1.55 2.436372271029366e-06 7.988845722374413e-07
1.65 4.3288471808299055e-06 8.288678072612618e-07
1.75 8.978984214121797e-06 8.725032747637349e-07
1.85 2.1513371013254955e-05 9.30887290652136e-07
1.95 5.784949838880617e-05 1.0057488058441292e-06


In [32]:
print(' MonoBQuadBF (d=d), 50x50 = ', CovMatrixSKA_joint[0][179,179],'\n','MonoBQuadBF (d=d) (d=d), 30x70 = ', CovMatrixSKA_joint[0][179+324,179+324] )
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][179,179+324],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][179+324,179] )  

 MonoBQuadBF (d=d), 50x50 =  1.9425535215326826e-06 
 MonoBQuadBF (d=d) (d=d), 30x70 =  4.3165840334060815e-06
 Monopole BB (d=d), 50x30 =  2.889255939893926e-06 
 Monopole BB (d=d), 30x50 =  2.889255939893926e-06


In [33]:
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][179,179+324], 
          np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][324+179,179])

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [34]:
len(CovMatrixSKA_50x50[0])

324

In [35]:
i_element = 288
j_element = i_element
l = len(CovMatrixSKA_50x50[0])

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  1.2444286340731441e-08 
 Element (d=d), 30x70 =  1.563436441154079e-08
 Element (d=d), 50x30 =  2.1011076877245555e-10 
 Element (d=d), 30x50 =  2.1011076877245555e-10 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [36]:
i_element = 56
j_element = i_element
l = len(CovMatrixSKA_50x50[0])

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  6.551780712795955e-08 
 Element (d=d), 30x70 =  1.6800215155336124e-07
 Element (d=d), 50x30 =  1.0369842895878622e-07 
 Element (d=d), 30x50 =  1.0369842895878622e-07 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [37]:
print(CovMatrixSKA_50x50[0][24,24], CovMatrixSKA_50x50[0][36,36])
print(CovMatrixSKA_50x50[0][24,36], CovMatrixSKA_50x50[0][36,24])

5.093876193886468e-07 1.0577598333198072e-06
5.353246240816097e-08 5.353246240816097e-08


In [38]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_50x50[k][10,25]**2 <= CovMatrixSKA_50x50[k][10,10]*CovMatrixSKA_50x50[k][25,25])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


In [39]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_30x70[k][18,30]**2 <= CovMatrixSKA_30x70[k][18,18]*CovMatrixSKA_30x70[k][30,30])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


## Signal-to-noise

In [40]:
splits = [2., 10/3.]

mySignals = [Signal(solver=Solver(zs=[0], **params_dict0), n_split=m) for m in splits]


In [41]:
d = np.linspace(20,160,36)
z_bins = np.linspace(0.15, 1.95, 19)

monopole_sigs = [mySignals[k]._monopole(d=d, z=z_bins) for k,_ in enumerate(splits)]

np.shape(monopole_sigs)

(2, 3, 19, 36)

In [42]:
monopoleBB_signal = [monopole_sigs[k][0] for k,_ in enumerate(splits)]

In [43]:
VarMONO_50x50 = [CovMatrixSKA_50x50[k,:36,:36] for k,_ in  enumerate(z_bins)]
VarMONO_30x70 = [CovMatrixSKA_30x70[k,:36,:36] for k,_ in  enumerate(z_bins)]
#VarMONO_10x90 = [CovMatrixSKA_10x90[k,:36,:36] for k,_ in  enumerate(z_bins)]

In [44]:
from fishermat import signal_to_noise

#covs = [ VarMONO_50x50, VarMONO_30x70, VarMONO_10x90 ]

covs = [ VarMONO_50x50, VarMONO_30x70 ]


cumSNR, SNRz = signal_to_noise(monopoleBB_signal[0], covs[0])


 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 58.77421213378112
Bin 2: 89.48345521707645
Bin 3: 114.76519820946098
Bin 4: 134.716436740866
Bin 5: 149.53509304639073
Bin 6: 159.5306944397853
Bin 7: 164.89350644693744
Bin 8: 165.8820114819602
Bin 9: 162.66901920794453
Bin 10: 155.44381004466567
Bin 11: 144.78901213682857
Bin 12: 131.1870337188347
Bin 13: 115.31104126390467
Bin 14: 98.24887356301002
Bin 15: 80.54138755170176
Bin 16: 63.4308923544652
Bin 17: 47.776283841079085
Bin 18: 34.10909687634405
Bin 19: 23.019044873900647

--------------------------------------------------

 Cumulative SNR : 521.0877057700761 

--------------------------------------------------


In [45]:
for k,_ in enumerate(splits):
    cumSNR, SNRz = signal_to_noise(monopoleBB_signal[k], covs[k])


 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 58.77421213378112
Bin 2: 89.48345521707645
Bin 3: 114.76519820946098
Bin 4: 134.716436740866
Bin 5: 149.53509304639073
Bin 6: 159.5306944397853
Bin 7: 164.89350644693744
Bin 8: 165.8820114819602
Bin 9: 162.66901920794453
Bin 10: 155.44381004466567
Bin 11: 144.78901213682857
Bin 12: 131.1870337188347
Bin 13: 115.31104126390467
Bin 14: 98.24887356301002
Bin 15: 80.54138755170176
Bin 16: 63.4308923544652
Bin 17: 47.776283841079085
Bin 18: 34.10909687634405
Bin 19: 23.019044873900647

--------------------------------------------------

 Cumulative SNR : 521.0877057700761 

--------------------------------------------------

 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 50.28

## Test Fisher Matrix

### Load the derivatives

In [46]:
with open('derivatives.pkl', 'rb') as File:
    derivatives_dict = pk.load(File)
    
derivatives_50x50 = derivatives_dict['split_50x50']
derivatives_30x70 = derivatives_dict['split_30x70']

In [47]:
dsignal_dsbias50 = derivatives_50x50['dsignal_dsbias']

dsignal_dgbias50_ = derivatives_50x50['dsignal_dgbias']

dsignal_dcosmic50 = derivatives_50x50['dsignal_dcosmic']

dsignal_dnum50 = derivatives_50x50['dsignal_dnumev']

np.shape(dsignal_dsbias50), np.shape(dsignal_dgbias50_), np.shape(dsignal_dcosmic50), np.shape(dsignal_dnum50)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [48]:
dsignal_dsbias30 = derivatives_30x70['dsignal_dsbias']

dsignal_dgbias30_ = derivatives_30x70['dsignal_dgbias']

dsignal_dcosmic30 = derivatives_30x70['dsignal_dcosmic']

dsignal_dnum30 = derivatives_30x70['dsignal_dnumev']

np.shape(dsignal_dsbias30), np.shape(dsignal_dgbias30_), np.shape(dsignal_dcosmic30), np.shape(dsignal_dnum30)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [49]:
dsignal_dtheta50 = np.concatenate([dsignal_dsbias50, dsignal_dnum50, dsignal_dgbias50_, dsignal_dcosmic50])

np.shape(dsignal_dtheta50)

(21, 19, 324)

In [50]:
dsignal_dtheta30 = np.concatenate([dsignal_dsbias30, dsignal_dnum30, dsignal_dgbias30_, dsignal_dcosmic30])

np.shape(dsignal_dtheta30)

(21, 19, 324)

In [51]:
dsignal_dsB50 = np.concatenate([dsignal_dsbias50[:4], np.zeros_like(dsignal_dsbias50[:4])], axis=2)
dsignal_dsB30 = np.concatenate([np.zeros_like(dsignal_dsbias30[:4]), dsignal_dsbias30[:4]], axis=2)

dsignal_dsM = np.concatenate([dsignal_dsbias50[4:], dsignal_dsbias30[4:]], axis=2)
dsignal_dnum = np.concatenate([dsignal_dnum50, dsignal_dnum30], axis=2)
dsignal_dgbias50 = np.concatenate([dsignal_dgbias50_, np.zeros_like(dsignal_dgbias50_[:4])], axis=2)
dsignal_dgbias30 = np.concatenate([np.zeros_like(dsignal_dgbias30_[:4]), dsignal_dgbias30_], axis=2)
dsignal_dcosmic = np.concatenate([dsignal_dcosmic50, dsignal_dcosmic30], axis=2)

In [52]:
dsignal_dtheta_multi = np.concatenate([dsignal_dsB50, dsignal_dsB30, dsignal_dsM, dsignal_dnum, dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dtheta_multi)

(29, 19, 648)

## Cosmic Parameters Fisher Matrices

In [53]:
Inv_CovMatrixSKA_joint = np.array([np.linalg.inv(CovMatrixSKA_joint[i]) for i,k in enumerate(z_bins)])

np.shape(Inv_CovMatrixSKA_50x50), np.shape(Inv_CovMatrixSKA_30x70), np.shape(Inv_CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 648, 648))

In [54]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(Inv_CovMatrixSKA_50x50[n], Inv_CovMatrixSKA_50x50[n].T, rtol=1e-4), 
          np.allclose(Inv_CovMatrixSKA_30x70[n], Inv_CovMatrixSKA_30x70[n].T, rtol=1e-4),
          np.allclose(Inv_CovMatrixSKA_joint[n], Inv_CovMatrixSKA_joint[n].T, rtol=1e-1))

0 True True True
1 True True True
2 True True True
3 True True True
4 True True True
5 True True True
6 True True True
7 True True True
8 True True True
9 True True True
10 True True True
11 True True True
12 True True True
13 True True True
14 True True True
15 True True True
16 True True True
17 True True True
18 True True True


In [55]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dcosmic50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dcosmic30, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dcosmic, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((5, 5), (5, 5), (5, 5))

In [56]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.579750899
496226.5426212941	96541.34153693542	269437.40309821675	-1928441.3798997358	1401221.6430270765
-19413664.00695389	-6055929.2249728255	-1928441.3798995449	80306511.07905662	-23831659.69196253
6641394.102308079	2108424.579750879	1401221.643026993	-23831659.691962913	13961632.845623164


In [57]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473726.474570977
647009.0811369446	138897.17270890815	310960.1564787712	-2499890.3801519116	1671688.8824865094
-22479456.449508905	-6988605.066998642	-2499890.3801519973	92043357.18543775	-28355007.711743876
7855159.798259621	2473726.4745712737	1671688.8824863958	-28355007.71174604	15915477.89555201


In [58]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

7644913.149420241	2466544.8861443447	885117.2351155959	-28721269.51251163	10160388.182646416
2466544.8861421505	846782.0439297776	195359.76275401967	-8895059.887975065	3155849.888833712
885117.2351277511	195359.76275964474	408644.89916158334	-3419560.9182246285	2271371.157701622
-28721269.512456253	-8895059.887951877	-3419560.9182172464	117063411.60683432	-36865421.954954304
10160388.182686161	3155849.888858996	2271371.157667018	-36865421.95510075	21276982.930528738


In [59]:
np.shape(dsignal_dcosmic50)

(5, 19, 324)

In [60]:
dsignal_dcosmic50_ = np.concatenate([dsignal_dcosmic50, np.zeros_like(dsignal_dcosmic30)], axis=2)
dsignal_dcosmic30_ = np.concatenate([np.zeros_like(dsignal_dcosmic50), dsignal_dcosmic30], axis=2)

dsignal_dcosmic_multi = np.concatenate([dsignal_dcosmic50_, dsignal_dcosmic30_])

np.shape(dsignal_dcosmic50_), np.shape(dsignal_dcosmic30_), np.shape(dsignal_dcosmic_multi)

((5, 19, 648), (5, 19, 648), (10, 19, 648))

In [61]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dcosmic_multi, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



In [62]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

34659716.274887726	13852063.148451775	-4772151.293549008	-123710478.64185812	27587764.43721887	-31526498.048359256	-12838759.906519623	5115236.204673875	111918308.0908535	-23243342.593047395
13852063.148452263	6111293.525039808	-3011964.2726371614	-45141570.97186677	10304037.322617095	-12846729.141351653	-5763654.005732943	3083290.906685285	41524546.46018055	-8947066.017020768
-4772151.293550826	-3011964.2726376797	3686237.2395266807	12863410.155256014	2095316.3992457523	5141521.553573232	3092903.4028123566	-3513590.4582826397	-14285676.803415973	-1130765.4726733116
-123710478.64186324	-45141570.97186752	12863410.155250624	521318820.95312965	-93462054.8340658	111904249.18204178	41490449.74598859	-14190131.944552569	-472690371.9947804	77775314.1762646
27587764.437210202	10304037.322613955	2095316.3992473485	-93462054.83402355	52277045.16251978	-23490661.642973777	-9027498.873392021	-1166404.2478173391	78669595.78238614	-42993256.64522915
-31526498.04837106	-12846729.141359922	5141521.55

In [63]:
np.linalg.det(FisherMatrixSKA_joint), np.linalg.det(FisherMatrixSKA_joint_)

(4.619906059311412e+29, 6.38608103064818e+62)

In [64]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in np.linalg.inv(FisherMatrixSKA_joint)]))

3.3603280811612336e-05	-7.170370659358787e-05	-2.9061038600187745e-05	2.685210966202224e-06	2.343546431612459e-06
-7.170370659377095e-05	0.00016475223791451616	6.990569047291119e-05	-5.049358747287124e-06	-6.407127784306303e-06
-2.9061038600599556e-05	6.990569047350411e-05	3.664206367719429e-05	-1.9252256389574677e-06	-3.738425449233605e-06
2.685210966183882e-06	-5.049358747231995e-06	-1.9252256388924435e-06	2.733647505232914e-07	1.458308890145434e-07
2.3435464316454493e-06	-6.4071277843444045e-06	-3.738425449212093e-06	1.4583088902063737e-07	5.299652244672289e-07


## Cosmic Parameters + Galaxy Bias FISHER MATRIX

In [65]:
dsignal_dpars50x50 = np.concatenate([dsignal_dgbias50_, dsignal_dcosmic50])
dsignal_dpars30x70 = np.concatenate([dsignal_dgbias30_, dsignal_dcosmic30])
dsignal_dpars_joint = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dpars50x50), np.shape(dsignal_dpars30x70), np.shape(dsignal_dpars_joint)

((9, 19, 324), (9, 19, 324), (13, 19, 648))

In [66]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dpars50x50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dpars30x70, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dpars_joint, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((9, 9), (9, 9), (13, 13))

In [67]:
np.linalg.det(FisherMatrixSKA_50x50), np.linalg.det(FisherMatrixSKA_30x70), np.linalg.det(FisherMatrixSKA_joint)

(2.978620645887647e+53, 6.949421003610326e+53, 9.999546056737581e+81)

In [68]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

25447240.777854692	7515169.987835224	-31713835.347808655	-9110094.934605015	2177814.361231719	1105856.1562454673	-932413.0557254107	-6618981.011715738	-2933471.9958783127
7515169.987835222	2808444.203160712	-9110094.934605015	-3225463.7597114304	1072997.0184735924	504090.3908093623	-332884.4190554553	-3396522.347946506	-474952.963605881
-31713835.347809885	-9110094.93460506	48443965.55355428	14405111.655087853	2321035.4568680176	305381.6409740199	1695881.1591249863	-10068274.409692686	11822762.336750662
-9110094.934605056	-3225463.7597113927	14405111.655087844	5400372.421185124	629545.2719306645	69738.45404022718	496145.3945584891	-2793044.5741037377	3003394.232552508
2177814.3612309718	1072997.0184736014	2321035.456869026	629545.2719306569	5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1105856.1562451809	504090.390809364	305381.6409744265	69738.45404022578	1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.57

In [69]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

33016651.384679236	8332488.267908776	-50360159.32634173	-12901331.246122913	1874549.3290753085	1015691.4324662106	-994719.3725617193	-5479385.917692468	-3961393.346895395
8332488.267908773	2733114.2927259943	-12901331.246122926	-4149358.7031630417	846945.2037477822	422633.8689456244	-337296.306303709	-2592730.776060814	-779727.1200034039
-50360159.32633923	-12901331.24612243	84500438.42434922	23254467.951169528	2067729.6980300494	117760.24214361416	1973312.5021055618	-9524773.356651565	13038348.119257785
-12901331.246122435	-4149358.7031629556	23254467.951169528	8118604.127277631	736809.8546538168	81090.78851551346	578643.8650978748	-3279873.2486417764	3469697.352632582
1874549.3290768762	846945.2037479944	2067729.6980268739	736809.8546534531	5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1015691.4324669703	422633.86894572224	117760.24214217364	81090.7885153587	1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473

In [70]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

40901032.969589815	11631362.711925276	-27640677.314019628	-7725485.339575319	-12705689.348332532	-3335432.769911933	-6195450.628962137	-2235202.3996309964	1543751.1068177726	825039.656899424	-784306.1211329706	-4543587.091185098	-3059471.1696748966
11631362.711925272	4237434.676572393	-7725485.339575325	-2644041.8254221324	-3335432.769911933	-1161371.5837176305	-2235202.399630997	-985248.4438474228	752448.4395950272	367187.64997519547	-274439.301242542	-2331193.2006142703	-591012.5382339006
-27640677.31406236	-7725485.339579337	57217100.4292124	16521181.194889681	-918461.3457856313	-514419.14445172023	-14314407.28959793	-3686203.330444675	1017643.5837422909	-72354.40503586925	1377121.2613394393	-5119237.609765647	9216544.037811866
-7725485.339579334	-2644041.8254225315	16521181.194889689	6087284.7828716	-514419.1444517205	-288550.3055072873	-3686203.330444673	-1253472.4153659076	275451.9670673846	-40529.37740008162	433960.794934831	-1471754.878659809	2399614.8109708885
-12705689.348259

In [71]:
dsignal_dpars_joint_ = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic_multi])

np.shape(dsignal_dpars_joint_)

(18, 19, 648)

In [72]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dpars_joint_, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 




 ... Success! ... 



In [73]:
np.shape(FisherMatrixSKA_joint_), np.linalg.det(FisherMatrixSKA_joint_)

((18, 18), 6.329299419835585e+113)

In [74]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

40901032.969589815	11631362.711925276	-27640677.314019628	-7725485.339575319	-12705689.348332532	-3335432.769911933	-6195450.628962137	-2235202.3996309964	20926527.486873463	9808588.548774984	-6302558.394362133	-66071195.57956721	7880790.634466715	-19382776.380055685	-8983548.891875561	5518252.273229162	61527608.48838214	-10940261.804141596
11631362.711925272	4237434.676572393	-7725485.339575325	-2644041.8254221324	-3335432.769911933	-1161371.5837176305	-2235202.399630997	-985248.4438474228	5976686.093074034	2789446.164087389	-1761765.202239025	-18904199.882961545	1847743.776674527	-5224237.653479005	-2422258.5141121913	1487325.9009964822	16573006.682347275	-2438756.3149084277
-27640677.31406236	-7725485.339579337	57217100.4292124	16521181.194889681	-918461.3457856313	-514419.14445172023	-14314407.28959793	-3686203.330444675	11299108.233019363	4156409.2373299105	57423.502228269535	-39896053.55177575	20123002.937174134	-10281464.649277057	-4228763.642365781	1319697.7591111718	34776815.9

# Joint Covariance Matrix 50x30 [CC & P]

## Check Cauchy-Schwarz inequality

The Cauchy-Schwarz inequality implies:

$$ \mathrm{COV}(X,Y) \leq \sqrt{\mathrm{VAR}(X)\cdot\mathrm{VAR}(Y)}$$

In [75]:
z_bins = np.array([0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])


In [76]:
CovMatrixSKA_joint = fm.cov_matrix_joint(d=dist[0], z_bins=z_bins, small_off_diagonal=True, rtol=1e-4, splits=[50,30], contributions='CC&P')


 ... Loading Covariance Matrices (Joint-splittings Analysis) [CC&P] [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 



In [77]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol = 1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [78]:
np.shape(CovMatrixSKA_50x50), np.shape(CovMatrixSKA_30x70), np.shape(CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 648, 648))

In [79]:
CovMatrixSKA_50x30 = np.array([CovMatrixSKA_joint[k][:324, 324:] for k,_ in enumerate(z_bins)])
CovMatrixSKA_30x50 = np.array([CovMatrixSKA_joint[k][324:, :324] for k,_ in enumerate(z_bins)])

In [80]:
np.shape(CovMatrixSKA_50x30), np.shape(CovMatrixSKA_30x50)

((19, 324, 324), (19, 324, 324))

In [81]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_50x30[n], CovMatrixSKA_30x50[n].T, rtol=1e-10), np.allclose(CovMatrixSKA_30x50[n], CovMatrixSKA_50x30[n].T, rtol = 1e-10))

0 True True
1 True True
2 True True
3 True True
4 True True
5 True True
6 True True
7 True True
8 True True
9 True True
10 True True
11 True True
12 True True
13 True True
14 True True
15 True True
16 True True
17 True True
18 True True


In [82]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol=1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [83]:
np.shape(CovMatrixSKA_50x30), np.shape(CovMatrixSKA_30x50)

((19, 324, 324), (19, 324, 324))

In [84]:
CovMatrixSKA_50x30[0,0,0], CovMatrixSKA_30x50[0,0,0]

(1.7591748859054704e-05, 1.7591748859054704e-05)

In [85]:
CovMatrixSKA_50x50[0,0,0], CovMatrixSKA_30x70[0,0,0]

(1.1406613424594502e-05, 2.728761334220845e-05)

In [86]:
CovMatrixSKA_50x30[0,17,267], CovMatrixSKA_30x50[0,267,17]

(-4.190495416728882e-09, -4.190495416728882e-09)

In [87]:
CovMatrixSKA_50x50[0,17,267], CovMatrixSKA_30x70[0,267,17]

(-4.188698962835081e-09, -5.405294849542528e-09)

## Checking directly from Joint Cov Matrix 50x30

Compare off diagonal elements (covariances) versus the correspondent diagonal ones (variances).

In [88]:
np.shape(CovMatrixSKA_joint), type(CovMatrixSKA_joint)

((19, 648, 648), list)

In [89]:
36*9

324

In [90]:
print(' Monopole BB (d=d), 50x50 = ', CovMatrixSKA_joint[0][0,0],'\n','Monopole BB (d=d), 30x70 = ', CovMatrixSKA_joint[0][324,324] ) 

 Monopole BB (d=d), 50x50 =  1.1406613424594502e-05 
 Monopole BB (d=d), 30x70 =  2.728761334220845e-05


In [91]:
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][0,324],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][324,0] ) 

 Monopole BB (d=d), 50x30 =  1.7591748859054704e-05 
 Monopole BB (d=d), 30x50 =  1.7591748859054704e-05


In [92]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][0,324], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][324,0])

0.15 True True
0.25 True True
0.35 True True
0.45 True True
0.55 True True
0.65 True True
0.75 True True
0.85 True True
0.95 True True
1.05 True True
1.15 True True
1.25 True True
1.35 True True
1.45 True True
1.55 True True
1.65 True True
1.75 True True
1.85 True True
1.95 True True


In [93]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]), CovMatrixSKA_joint[k][0,324])

0.15 1.764254110592856e-05 1.7591748859054704e-05
0.25 7.014350303522421e-06 6.987339425132502e-06
0.35 3.904079159778656e-06 3.882715928743635e-06
0.45 2.5784765980908824e-06 2.5575038345243217e-06
0.55 1.8931986511136867e-06 1.869650246493458e-06
0.65 1.4969289169443364e-06 1.4684628360388927e-06
0.75 1.2532721988093958e-06 1.2167672009639408e-06
0.85 1.1007803772214066e-06 1.0519094735961752e-06
0.95 1.0104386161984314e-06 9.422589984695888e-07
1.05 9.703620669081107e-07 8.712130185385997e-07
1.15 9.797925594098938e-07 8.309484737703343e-07
1.25 1.0534917202039827e-06 8.215568760701692e-07
1.35 1.2326007500359773e-06 8.554091832340599e-07
1.45 1.6096843827924573e-06 9.7033160666954e-07
1.55 2.436372271029366e-06 1.2809708253482055e-06
1.65 4.3288471808299055e-06 2.099426305851096e-06
1.75 8.978984214121797e-06 4.330763768587618e-06
1.85 2.1513371013254955e-05 1.0823454774754264e-05
1.95 5.784949838880617e-05 3.0686479427370136e-05


In [94]:
print(' MonoBQuadBF (d=d), 50x50 = ', CovMatrixSKA_joint[0][179,179],'\n','MonoBQuadBF (d=d) (d=d), 30x70 = ', CovMatrixSKA_joint[0][179+324,179+324] )
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][179,179+324],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][179+324,179] )  

 MonoBQuadBF (d=d), 50x50 =  1.9425535215326826e-06 
 MonoBQuadBF (d=d) (d=d), 30x70 =  4.3165840334060815e-06
 Monopole BB (d=d), 50x30 =  2.8892575230477216e-06 
 Monopole BB (d=d), 30x50 =  2.8892575230477216e-06


In [95]:
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][179,179+324], 
          np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][324+179,179])

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [96]:
len(CovMatrixSKA_50x50[0])

324

In [97]:
i_element = 288
j_element = i_element
l = len(CovMatrixSKA_50x50[0])

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  1.2444286340731441e-08 
 Element (d=d), 30x70 =  1.563436441154079e-08
 Element (d=d), 50x30 =  2.607716902285996e-10 
 Element (d=d), 30x50 =  2.607716902285996e-10 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [98]:
i_element = 56
j_element = i_element
l = len(CovMatrixSKA_50x50[0])

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  6.551780712795955e-08 
 Element (d=d), 30x70 =  1.6800215155336124e-07
 Element (d=d), 50x30 =  1.0369871844976598e-07 
 Element (d=d), 30x50 =  1.0369871844976598e-07 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [99]:
print(CovMatrixSKA_50x50[0][24,24], CovMatrixSKA_50x50[0][36,36])
print(CovMatrixSKA_50x50[0][24,36], CovMatrixSKA_50x50[0][36,24])

5.093876193886468e-07 1.0577598333198072e-06
5.353246240816097e-08 5.353246240816097e-08


In [100]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_50x50[k][10,25]**2 <= CovMatrixSKA_50x50[k][10,10]*CovMatrixSKA_50x50[k][25,25])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


In [101]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_30x70[k][18,30]**2 <= CovMatrixSKA_30x70[k][18,18]*CovMatrixSKA_30x70[k][30,30])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


## Signal-to-noise

In [102]:
splits = [2., 10/3.]

mySignals = [Signal(solver=Solver(zs=[0], **params_dict0), n_split=m) for m in splits]


In [103]:
d = np.linspace(20,160,36)
z_bins = np.linspace(0.15, 1.95, 19)

monopole_sigs = [mySignals[k]._monopole(d=d, z=z_bins) for k,_ in enumerate(splits)]

np.shape(monopole_sigs)

(2, 3, 19, 36)

In [104]:
monopoleBB_signal = [monopole_sigs[k][0] for k,_ in enumerate(splits)]

In [105]:
VarMONO_50x50 = [CovMatrixSKA_50x50[k,:36,:36] for k,_ in  enumerate(z_bins)]
VarMONO_30x70 = [CovMatrixSKA_30x70[k,:36,:36] for k,_ in  enumerate(z_bins)]
#VarMONO_10x90 = [CovMatrixSKA_10x90[k,:36,:36] for k,_ in  enumerate(z_bins)]

In [106]:

#covs = [ VarMONO_50x50, VarMONO_30x70, VarMONO_10x90 ]

covs = [ VarMONO_50x50, VarMONO_30x70 ]


cumSNR, SNRz = fm.signal_to_noise(monopoleBB_signal[0], covs[0])


 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 58.77421213378112
Bin 2: 89.48345521707645
Bin 3: 114.76519820946098
Bin 4: 134.716436740866
Bin 5: 149.53509304639073
Bin 6: 159.5306944397853
Bin 7: 164.89350644693744
Bin 8: 165.8820114819602
Bin 9: 162.66901920794453
Bin 10: 155.44381004466567
Bin 11: 144.78901213682857
Bin 12: 131.1870337188347
Bin 13: 115.31104126390467
Bin 14: 98.24887356301002
Bin 15: 80.54138755170176
Bin 16: 63.4308923544652
Bin 17: 47.776283841079085
Bin 18: 34.10909687634405
Bin 19: 23.019044873900647

--------------------------------------------------

 Cumulative SNR : 521.0877057700761 

--------------------------------------------------


In [107]:
for k,_ in enumerate(splits):
    cumSNR, SNRz = fm.signal_to_noise(monopoleBB_signal[k], covs[k])


 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 58.77421213378112
Bin 2: 89.48345521707645
Bin 3: 114.76519820946098
Bin 4: 134.716436740866
Bin 5: 149.53509304639073
Bin 6: 159.5306944397853
Bin 7: 164.89350644693744
Bin 8: 165.8820114819602
Bin 9: 162.66901920794453
Bin 10: 155.44381004466567
Bin 11: 144.78901213682857
Bin 12: 131.1870337188347
Bin 13: 115.31104126390467
Bin 14: 98.24887356301002
Bin 15: 80.54138755170176
Bin 16: 63.4308923544652
Bin 17: 47.776283841079085
Bin 18: 34.10909687634405
Bin 19: 23.019044873900647

--------------------------------------------------

 Cumulative SNR : 521.0877057700761 

--------------------------------------------------

 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 50.28

## Test Fisher Matrix

### Load the derivatives

In [108]:
with open('derivatives.pkl', 'rb') as File:
    derivatives_dict = pk.load(File)
    
derivatives_50x50 = derivatives_dict['split_50x50']
derivatives_30x70 = derivatives_dict['split_30x70']

In [109]:
dsignal_dsbias50 = derivatives_50x50['dsignal_dsbias']

dsignal_dgbias50_ = derivatives_50x50['dsignal_dgbias']

dsignal_dcosmic50 = derivatives_50x50['dsignal_dcosmic']

dsignal_dnum50 = derivatives_50x50['dsignal_dnumev']

np.shape(dsignal_dsbias50), np.shape(dsignal_dgbias50_), np.shape(dsignal_dcosmic50), np.shape(dsignal_dnum50)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [110]:
dsignal_dsbias30 = derivatives_30x70['dsignal_dsbias']

dsignal_dgbias30_ = derivatives_30x70['dsignal_dgbias']

dsignal_dcosmic30 = derivatives_30x70['dsignal_dcosmic']

dsignal_dnum30 = derivatives_30x70['dsignal_dnumev']

np.shape(dsignal_dsbias30), np.shape(dsignal_dgbias30_), np.shape(dsignal_dcosmic30), np.shape(dsignal_dnum30)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [111]:
dsignal_dtheta50 = np.concatenate([dsignal_dsbias50, dsignal_dnum50, dsignal_dgbias50_, dsignal_dcosmic50])

np.shape(dsignal_dtheta50)

(21, 19, 324)

In [112]:
dsignal_dtheta30 = np.concatenate([dsignal_dsbias30, dsignal_dnum30, dsignal_dgbias30_, dsignal_dcosmic30])

np.shape(dsignal_dtheta30)

(21, 19, 324)

In [113]:
dsignal_dsB50 = np.concatenate([dsignal_dsbias50[:4], np.zeros_like(dsignal_dsbias50[:4])], axis=2)
dsignal_dsB30 = np.concatenate([np.zeros_like(dsignal_dsbias30[:4]), dsignal_dsbias30[:4]], axis=2)

dsignal_dsM = np.concatenate([dsignal_dsbias50[4:], dsignal_dsbias30[4:]], axis=2)
dsignal_dnum = np.concatenate([dsignal_dnum50, dsignal_dnum30], axis=2)
dsignal_dgbias50 = np.concatenate([dsignal_dgbias50_, np.zeros_like(dsignal_dgbias50_[:4])], axis=2)
dsignal_dgbias30 = np.concatenate([np.zeros_like(dsignal_dgbias30_[:4]), dsignal_dgbias30_], axis=2)
dsignal_dcosmic = np.concatenate([dsignal_dcosmic50, dsignal_dcosmic30], axis=2)

In [114]:
dsignal_dtheta_multi = np.concatenate([dsignal_dsB50, dsignal_dsB30, dsignal_dsM, dsignal_dnum, dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dtheta_multi)

(29, 19, 648)

## Cosmic Parameters Fisher Matrices

In [115]:
Inv_CovMatrixSKA_joint = np.array([np.linalg.inv(CovMatrixSKA_joint[i]) for i,k in enumerate(z_bins)])

np.shape(Inv_CovMatrixSKA_50x50), np.shape(Inv_CovMatrixSKA_30x70), np.shape(Inv_CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 648, 648))

In [116]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(Inv_CovMatrixSKA_50x50[n], Inv_CovMatrixSKA_50x50[n].T, rtol=1e-4), 
          np.allclose(Inv_CovMatrixSKA_30x70[n], Inv_CovMatrixSKA_30x70[n].T, rtol=1e-4),
          np.allclose(Inv_CovMatrixSKA_joint[n], Inv_CovMatrixSKA_joint[n].T, rtol=1e-1))

0 True True True
1 True True True
2 True True True
3 True True True
4 True True True
5 True True True
6 True True True
7 True True True
8 True True True
9 True True True
10 True True True
11 True True True
12 True True True
13 True True True
14 True True True
15 True True True
16 True True True
17 True True True
18 True True True


In [117]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dcosmic50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dcosmic30, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dcosmic, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((5, 5), (5, 5), (5, 5))

In [118]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.579750899
496226.5426212941	96541.34153693542	269437.40309821675	-1928441.3798997358	1401221.6430270765
-19413664.00695389	-6055929.2249728255	-1928441.3798995449	80306511.07905662	-23831659.69196253
6641394.102308079	2108424.579750879	1401221.643026993	-23831659.691962913	13961632.845623164


In [119]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473726.474570977
647009.0811369446	138897.17270890815	310960.1564787712	-2499890.3801519116	1671688.8824865094
-22479456.449508905	-6988605.066998642	-2499890.3801519973	92043357.18543775	-28355007.711743876
7855159.798259621	2473726.4745712737	1671688.8824863958	-28355007.71174604	15915477.89555201


In [120]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

7393050.977918177	2383284.3220618693	863475.6507951431	-27777333.960281923	9935262.070098698
2383284.3220597464	817193.9654672426	191980.80674421845	-8597227.924881103	3088642.5653488343
863475.6507987647	191980.8067444012	394337.56680249213	-3333277.289401583	2206390.2229661164
-27777333.960260592	-8597227.924874905	-3333277.289401003	113184709.17958054	-36053133.9668886
9935262.070082664	3088642.5653375383	2206390.222975766	-36053133.96687589	20801956.492821


In [121]:
np.shape(dsignal_dcosmic50)

(5, 19, 324)

In [122]:
dsignal_dcosmic50_ = np.concatenate([dsignal_dcosmic50, np.zeros_like(dsignal_dcosmic30)], axis=2)
dsignal_dcosmic30_ = np.concatenate([np.zeros_like(dsignal_dcosmic50), dsignal_dcosmic30], axis=2)

dsignal_dcosmic_multi = np.concatenate([dsignal_dcosmic50_, dsignal_dcosmic30_])

np.shape(dsignal_dcosmic50_), np.shape(dsignal_dcosmic30_), np.shape(dsignal_dcosmic_multi)

((5, 19, 648), (5, 19, 648), (10, 19, 648))

In [123]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dcosmic_multi, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



In [124]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

35268641.169105165	14073965.903849546	-4790828.809878196	-125964392.87725894	28078415.02125295	-32265951.04235994	-13103486.3772117	5121471.223890136	114661915.6889265	-23852665.565543037
14073965.903850356	6196705.384660047	-3024875.7813160364	-45925004.72450679	10489882.777632276	-13111437.05989921	-5864090.5405076435	3093888.2264252338	42461758.74229238	-9168884.645423714
-4790828.809879368	-3024875.7813163386	3705209.329538938	12925584.697688058	2120614.6347166835	5147703.084567771	3103480.6386359725	-3539644.0060946676	-14298849.38761549	-1188171.085717522
-125964392.87723409	-45925004.724495105	12925584.697683452	530611769.1232791	-95113095.70066044	114647679.0389023	42427684.760702886	-14203568.698204804	-483986332.2597481	79853049.4926907
28078415.02125742	10489882.77763418	2120614.6347155264	-95113095.70071512	52988707.51662617	-24107356.11161408	-9251494.357591338	-1225453.6531985896	80776011.34982698	-43945652.94553769
-32265951.042416684	-13111437.059909951	5147703.08454712

In [125]:
np.linalg.det(FisherMatrixSKA_joint), np.linalg.det(FisherMatrixSKA_joint_)

(3.80170114158023e+29, 5.799576974802569e+62)

In [126]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in np.linalg.inv(FisherMatrixSKA_joint)]))

3.503860778506977e-05	-7.487103977861287e-05	-3.0310389876525967e-05	2.799271210707713e-06	2.448383993150621e-06
-7.487103977874395e-05	0.0001723020078428505	7.301900175509824e-05	-5.273294698966217e-06	-6.708173132462623e-06
-3.0310389877143207e-05	7.301900175616668e-05	3.817636217270933e-05	-2.0092782147032092e-06	-3.896780524120084e-06
2.799271210699743e-06	-5.27329469893763e-06	-2.0092782146407583e-06	2.846888268087113e-07	1.5253399862127172e-07
2.448383993220435e-06	-6.708173132583796e-06	-3.896780524148196e-06	1.5253399862838547e-07	5.523980204791495e-07


## Cosmic Parameters + Galaxy Bias FISHER MATRIX

In [127]:
dsignal_dpars50x50 = np.concatenate([dsignal_dgbias50_, dsignal_dcosmic50])
dsignal_dpars30x70 = np.concatenate([dsignal_dgbias30_, dsignal_dcosmic30])
dsignal_dpars_joint = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dpars50x50), np.shape(dsignal_dpars30x70), np.shape(dsignal_dpars_joint)

((9, 19, 324), (9, 19, 324), (13, 19, 648))

In [128]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dpars50x50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dpars30x70, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dpars_joint, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((9, 9), (9, 9), (13, 13))

In [129]:
np.linalg.det(FisherMatrixSKA_50x50), np.linalg.det(FisherMatrixSKA_30x70), np.linalg.det(FisherMatrixSKA_joint)

(2.978620645887647e+53, 6.949421003610326e+53, 1.0764848219840307e+82)

In [130]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

25447240.777854692	7515169.987835224	-31713835.347808655	-9110094.934605015	2177814.361231719	1105856.1562454673	-932413.0557254107	-6618981.011715738	-2933471.9958783127
7515169.987835222	2808444.203160712	-9110094.934605015	-3225463.7597114304	1072997.0184735924	504090.3908093623	-332884.4190554553	-3396522.347946506	-474952.963605881
-31713835.347809885	-9110094.93460506	48443965.55355428	14405111.655087853	2321035.4568680176	305381.6409740199	1695881.1591249863	-10068274.409692686	11822762.336750662
-9110094.934605056	-3225463.7597113927	14405111.655087844	5400372.421185124	629545.2719306645	69738.45404022718	496145.3945584891	-2793044.5741037377	3003394.232552508
2177814.3612309718	1072997.0184736014	2321035.456869026	629545.2719306569	5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1105856.1562451809	504090.390809364	305381.6409744265	69738.45404022578	1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.57

In [131]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

33016651.384679236	8332488.267908776	-50360159.32634173	-12901331.246122913	1874549.3290753085	1015691.4324662106	-994719.3725617193	-5479385.917692468	-3961393.346895395
8332488.267908773	2733114.2927259943	-12901331.246122926	-4149358.7031630417	846945.2037477822	422633.8689456244	-337296.306303709	-2592730.776060814	-779727.1200034039
-50360159.32633923	-12901331.24612243	84500438.42434922	23254467.951169528	2067729.6980300494	117760.24214361416	1973312.5021055618	-9524773.356651565	13038348.119257785
-12901331.246122435	-4149358.7031629556	23254467.951169528	8118604.127277631	736809.8546538168	81090.78851551346	578643.8650978748	-3279873.2486417764	3469697.352632582
1874549.3290768762	846945.2037479944	2067729.6980268739	736809.8546534531	5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1015691.4324669703	422633.86894572224	117760.24214217364	81090.7885153587	1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473

In [132]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

45056115.04321487	12868503.391986465	-30913266.777216937	-8679538.271205634	-22621884.742832616	-6073643.848014291	7529802.113562533	1620330.0838263158	1277229.2764293447	696074.5689991552	-691526.3196927829	-3709565.2925022533	-2727422.6152514825
12868503.391986463	4690256.864039837	-8679538.271205636	-2943180.299467973	-6073643.848014291	-2080191.2994007873	1620330.0838263105	252539.23417912755	651826.2945108879	319349.1778817093	-242019.39917556048	-2014288.203864682	-512643.84782449616
-30913266.777202196	-8679538.271204118	60630855.62803891	17750461.621151518	8767118.920199964	2232329.990251182	-29305063.565480813	-8397956.20431456	1188816.7070869617	32909.02060687145	1249697.9122478666	-5576288.517081196	8667876.033348091
-8679538.271204118	-2943180.299467778	17750461.621151518	6606170.468578394	2232329.990251178	624274.2414625247	-8397956.204314565	-3038156.6814630073	312865.9876076333	-11420.199872686713	387967.8190757968	-1548457.7027625232	2222563.910087663
-22621884.74286408

In [133]:
dsignal_dpars_joint_ = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic_multi])

np.shape(dsignal_dpars_joint_)

(18, 19, 648)

In [134]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dpars_joint_, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 




 ... Success! ... 



In [135]:
np.shape(FisherMatrixSKA_joint_), np.linalg.det(FisherMatrixSKA_joint_)

((18, 18), 8.388974487012565e+113)

In [136]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

45056115.04321487	12868503.391986465	-30913266.777216937	-8679538.271205634	-22621884.742832616	-6073643.848014291	7529802.113562533	1620330.0838263158	21468198.412166715	10043128.063477222	-6409607.5211132085	-67866577.77817763	8175652.51016016	-20190969.135737356	-9347053.494478084	5718081.2014204115	64157012.48567533	-10903075.125411658
12868503.391986463	4690256.864039837	-8679538.271205636	-2943180.299467973	-6073643.848014291	-2080191.2994007873	1620330.0838263105	252539.23417912755	6221154.57750778	2892740.9017117615	-1802813.9942347999	-19727000.159026258	1964069.6066690662	-5569328.282996887	-2573391.7238300545	1560794.5950592398	17712711.955161545	-2476713.45449357
-30913266.777202196	-8679538.271204118	60630855.62803891	17750461.621151518	8767118.920199964	2232329.990251182	-29305063.565480813	-8397956.20431456	11459249.043103997	4191557.065063723	124490.06102099083	-40581879.555599	20419935.27392009	-10270432.336017016	-4158648.044456861	1125207.8512268793	35005591.03851783

# Joint Covariance Matrix 50x30 [CC & CP]

## Check Cauchy-Schwarz inequality 

The Cauchy-Schwarz inequality implies:

$$ \mathrm{COV}(X,Y) \leq \sqrt{\mathrm{VAR}(X)\cdot\mathrm{VAR}(Y)}$$

In [137]:
z_bins = np.array([0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])


In [138]:
CovMatrixSKA_joint = fm.cov_matrix_joint(d=dist[0], z_bins=z_bins, small_off_diagonal=True, rtol=1e-4, splits=[50,30], contributions='CC&CP')


 ... Loading Covariance Matrices (Joint-splittings Analysis) [CC&CP] [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 



In [139]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol = 1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [140]:
np.shape(CovMatrixSKA_50x50), np.shape(CovMatrixSKA_30x70), np.shape(CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 648, 648))

In [141]:
CovMatrixSKA_50x30 = np.array([CovMatrixSKA_joint[k][:324, 324:] for k,_ in enumerate(z_bins)])
CovMatrixSKA_30x50 = np.array([CovMatrixSKA_joint[k][324:, :324] for k,_ in enumerate(z_bins)])

In [142]:
np.shape(CovMatrixSKA_50x30), np.shape(CovMatrixSKA_30x50)

((19, 324, 324), (19, 324, 324))

In [143]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_50x30[n], CovMatrixSKA_30x50[n].T, rtol=1e-10), np.allclose(CovMatrixSKA_30x50[n], CovMatrixSKA_50x30[n].T, rtol = 1e-10))

0 True True
1 True True
2 True True
3 True True
4 True True
5 True True
6 True True
7 True True
8 True True
9 True True
10 True True
11 True True
12 True True
13 True True
14 True True
15 True True
16 True True
17 True True
18 True True


In [144]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol=1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [145]:
np.shape(CovMatrixSKA_50x30), np.shape(CovMatrixSKA_30x50)

((19, 324, 324), (19, 324, 324))

In [146]:
CovMatrixSKA_50x30[0,0,0], CovMatrixSKA_30x50[0,0,0]

(1.7610798515470308e-05, 1.7610798515470308e-05)

In [147]:
CovMatrixSKA_50x50[0,0,0], CovMatrixSKA_30x70[0,0,0]

(1.1406613424594502e-05, 2.728761334220845e-05)

In [148]:
CovMatrixSKA_50x30[0,17,267], CovMatrixSKA_30x50[0,267,17]

(-4.188698962835081e-09, -4.188698962835081e-09)

In [149]:
CovMatrixSKA_50x50[0,17,267], CovMatrixSKA_30x70[0,267,17]

(-4.188698962835081e-09, -5.405294849542528e-09)

## Checking directly from Joint Cov Matrix 50x30

Compare off diagonal elements (covariances) versus the correspondent diagonal ones (variances).

In [150]:
np.shape(CovMatrixSKA_joint), type(CovMatrixSKA_joint)

((19, 648, 648), list)

In [151]:
36*9

324

In [152]:
print(' Monopole BB (d=d), 50x50 = ', CovMatrixSKA_joint[0][0,0],'\n','Monopole BB (d=d), 30x70 = ', CovMatrixSKA_joint[0][324,324] ) 

 Monopole BB (d=d), 50x50 =  1.1406613424594502e-05 
 Monopole BB (d=d), 30x70 =  2.728761334220845e-05


In [153]:
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][0,324],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][324,0] ) 

 Monopole BB (d=d), 50x30 =  1.7610798515470308e-05 
 Monopole BB (d=d), 30x50 =  1.7610798515470308e-05


In [154]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][0,324], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][324,0])

0.15 True True
0.25 True True
0.35 True True
0.45 True True
0.55 True True
0.65 True True
0.75 True True
0.85 True True
0.95 True True
1.05 True True
1.15 True True
1.25 True True
1.35 True True
1.45 True True
1.55 True True
1.65 True True
1.75 True True
1.85 True True
1.95 True True


In [155]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]), CovMatrixSKA_joint[k][0,324])

0.15 1.764254110592856e-05 1.7610798515470308e-05
0.25 7.014350303522421e-06 7.000400277266332e-06
0.35 3.904079159778656e-06 3.895062136086899e-06
0.45 2.5784765980908824e-06 2.57107636566038e-06
0.55 1.8931986511136867e-06 1.8859244727706914e-06
0.65 1.4969289169443364e-06 1.4887901541805723e-06
0.75 1.2532721988093958e-06 1.2431276615296576e-06
0.85 1.1007803772214066e-06 1.0870014862248079e-06
0.95 1.0104386161984314e-06 9.901499630882853e-07
1.05 9.703620669081107e-07 9.379489200039358e-07
1.15 9.797925594098938e-07 9.242245765777368e-07
1.25 1.0534917202039827e-06 9.507403503374526e-07
1.35 1.2326007500359773e-06 1.0273878979568094e-06
1.45 1.6096843827924573e-06 1.1732188677180108e-06
1.55 2.436372271029366e-06 1.432633277938617e-06
1.65 4.3288471808299055e-06 1.8786578035440482e-06
1.75 8.978984214121797e-06 2.65237939130472e-06
1.85 2.1513371013254955e-05 4.045155132628991e-06
1.95 5.784949838880617e-05 6.620985876809156e-06


In [156]:
print(' MonoBQuadBF (d=d), 50x50 = ', CovMatrixSKA_joint[0][179,179],'\n','MonoBQuadBF (d=d) (d=d), 30x70 = ', CovMatrixSKA_joint[0][179+324,179+324] )
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][179,179+324],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][179+324,179] )  

 MonoBQuadBF (d=d), 50x50 =  1.9425535215326826e-06 
 MonoBQuadBF (d=d) (d=d), 30x70 =  4.3165840334060815e-06
 Monopole BB (d=d), 50x30 =  2.8911929143650812e-06 
 Monopole BB (d=d), 30x50 =  2.8911929143650812e-06


In [157]:
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][179,179+324], 
          np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][324+179,179])

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [158]:
len(CovMatrixSKA_50x50[0])

324

In [159]:
i_element = 288
j_element = i_element
l = len(CovMatrixSKA_50x50[0])

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  1.2444286340731441e-08 
 Element (d=d), 30x70 =  1.563436441154079e-08
 Element (d=d), 50x30 =  1.0220284535632892e-08 
 Element (d=d), 30x50 =  1.0220284535632892e-08 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [160]:
i_element = 56
j_element = i_element
l = len(CovMatrixSKA_50x50[0])

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  6.551780712795955e-08 
 Element (d=d), 30x70 =  1.6800215155336124e-07
 Element (d=d), 50x30 =  1.0388496607481709e-07 
 Element (d=d), 30x50 =  1.0388496607481709e-07 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [161]:
print(CovMatrixSKA_50x50[0][24,24], CovMatrixSKA_50x50[0][36,36])
print(CovMatrixSKA_50x50[0][24,36], CovMatrixSKA_50x50[0][36,24])

5.093876193886468e-07 1.0577598333198072e-06
5.353246240816097e-08 5.353246240816097e-08


In [162]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_50x50[k][10,25]**2 <= CovMatrixSKA_50x50[k][10,10]*CovMatrixSKA_50x50[k][25,25])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


In [163]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_30x70[k][18,30]**2 <= CovMatrixSKA_30x70[k][18,18]*CovMatrixSKA_30x70[k][30,30])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


## Signal-to-noise

In [164]:
splits = [2., 10/3.]

mySignals = [Signal(solver=Solver(zs=[0], **params_dict0), n_split=m) for m in splits]


In [165]:
d = np.linspace(20,160,36)
z_b = np.linspace(0.15, 1.95, 19)

monopole_sigs = [mySignals[k]._monopole(d=d, z=z_b) for k,_ in enumerate(splits)]

np.shape(monopole_sigs)

(2, 3, 19, 36)

In [166]:
monopoleBB_signal = [monopole_sigs[k][0] for k,_ in enumerate(splits)]

In [167]:
VarMONO_50x50 = [CovMatrixSKA_50x50[k,:36,:36] for k,_ in  enumerate(z_b)]
VarMONO_30x70 = [CovMatrixSKA_30x70[k,:36,:36] for k,_ in  enumerate(z_b)]
#VarMONO_10x90 = [CovMatrixSKA_10x90[k,:36,:36] for k,_ in  enumerate(z_bins)]

In [168]:

#covs = [ VarMONO_50x50, VarMONO_30x70, VarMONO_10x90 ]

covs = [ VarMONO_50x50, VarMONO_30x70 ]


cumSNR, SNRz = fm.signal_to_noise(monopoleBB_signal[0], covs[0])


 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 58.77421213378112
Bin 2: 89.48345521707645
Bin 3: 114.76519820946098
Bin 4: 134.716436740866
Bin 5: 149.53509304639073
Bin 6: 159.5306944397853
Bin 7: 164.89350644693744
Bin 8: 165.8820114819602
Bin 9: 162.66901920794453
Bin 10: 155.44381004466567
Bin 11: 144.78901213682857
Bin 12: 131.1870337188347
Bin 13: 115.31104126390467
Bin 14: 98.24887356301002
Bin 15: 80.54138755170176
Bin 16: 63.4308923544652
Bin 17: 47.776283841079085
Bin 18: 34.10909687634405
Bin 19: 23.019044873900647

--------------------------------------------------

 Cumulative SNR : 521.0877057700761 

--------------------------------------------------


In [169]:
for k,_ in enumerate(splits):
    cumSNR, SNRz = fm.signal_to_noise(monopoleBB_signal[k], covs[k])


 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 58.77421213378112
Bin 2: 89.48345521707645
Bin 3: 114.76519820946098
Bin 4: 134.716436740866
Bin 5: 149.53509304639073
Bin 6: 159.5306944397853
Bin 7: 164.89350644693744
Bin 8: 165.8820114819602
Bin 9: 162.66901920794453
Bin 10: 155.44381004466567
Bin 11: 144.78901213682857
Bin 12: 131.1870337188347
Bin 13: 115.31104126390467
Bin 14: 98.24887356301002
Bin 15: 80.54138755170176
Bin 16: 63.4308923544652
Bin 17: 47.776283841079085
Bin 18: 34.10909687634405
Bin 19: 23.019044873900647

--------------------------------------------------

 Cumulative SNR : 521.0877057700761 

--------------------------------------------------

 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 50.28

## Test Fisher Matrix

### Load the derivatives

In [170]:
with open('derivatives.pkl', 'rb') as File:
    derivatives_dict = pk.load(File)
    
derivatives_50x50 = derivatives_dict['split_50x50']
derivatives_30x70 = derivatives_dict['split_30x70']

In [171]:
dsignal_dsbias50 = derivatives_50x50['dsignal_dsbias']

dsignal_dgbias50_ = derivatives_50x50['dsignal_dgbias']

dsignal_dcosmic50 = derivatives_50x50['dsignal_dcosmic']

dsignal_dnum50 = derivatives_50x50['dsignal_dnumev']

np.shape(dsignal_dsbias50), np.shape(dsignal_dgbias50_), np.shape(dsignal_dcosmic50), np.shape(dsignal_dnum50)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [172]:
dsignal_dsbias30 = derivatives_30x70['dsignal_dsbias']

dsignal_dgbias30_ = derivatives_30x70['dsignal_dgbias']

dsignal_dcosmic30 = derivatives_30x70['dsignal_dcosmic']

dsignal_dnum30 = derivatives_30x70['dsignal_dnumev']

np.shape(dsignal_dsbias30), np.shape(dsignal_dgbias30_), np.shape(dsignal_dcosmic30), np.shape(dsignal_dnum30)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [173]:
dsignal_dtheta50 = np.concatenate([dsignal_dsbias50, dsignal_dnum50, dsignal_dgbias50_, dsignal_dcosmic50])

np.shape(dsignal_dtheta50)

(21, 19, 324)

In [174]:
dsignal_dtheta30 = np.concatenate([dsignal_dsbias30, dsignal_dnum30, dsignal_dgbias30_, dsignal_dcosmic30])

np.shape(dsignal_dtheta30)

(21, 19, 324)

In [175]:
dsignal_dsB50 = np.concatenate([dsignal_dsbias50[:4], np.zeros_like(dsignal_dsbias50[:4])], axis=2)
dsignal_dsB30 = np.concatenate([np.zeros_like(dsignal_dsbias30[:4]), dsignal_dsbias30[:4]], axis=2)

dsignal_dsM = np.concatenate([dsignal_dsbias50[4:], dsignal_dsbias30[4:]], axis=2)
dsignal_dnum = np.concatenate([dsignal_dnum50, dsignal_dnum30], axis=2)
dsignal_dgbias50 = np.concatenate([dsignal_dgbias50_, np.zeros_like(dsignal_dgbias50_[:4])], axis=2)
dsignal_dgbias30 = np.concatenate([np.zeros_like(dsignal_dgbias30_[:4]), dsignal_dgbias30_], axis=2)
dsignal_dcosmic = np.concatenate([dsignal_dcosmic50, dsignal_dcosmic30], axis=2)

In [176]:
dsignal_dtheta_multi = np.concatenate([dsignal_dsB50, dsignal_dsB30, dsignal_dsM, dsignal_dnum, dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dtheta_multi)

(29, 19, 648)

## Cosmic Parameters Fisher Matrices

In [177]:
Inv_CovMatrixSKA_joint = np.array([np.linalg.inv(CovMatrixSKA_joint[i]) for i,k in enumerate(z_bins)])

np.shape(Inv_CovMatrixSKA_50x50), np.shape(Inv_CovMatrixSKA_30x70), np.shape(Inv_CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 648, 648))

In [178]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(Inv_CovMatrixSKA_50x50[n], Inv_CovMatrixSKA_50x50[n].T, rtol=1e-4), 
          np.allclose(Inv_CovMatrixSKA_30x70[n], Inv_CovMatrixSKA_30x70[n].T, rtol=1e-4),
          np.allclose(Inv_CovMatrixSKA_joint[n], Inv_CovMatrixSKA_joint[n].T, rtol=1e-2))

0 True True True
1 True True True
2 True True False
3 True True True
4 True True True
5 True True True
6 True True True
7 True True True
8 True True True
9 True True True
10 True True True
11 True True True
12 True True True
13 True True True
14 True True True
15 True True True
16 True True True
17 True True True
18 True True True


In [179]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dcosmic50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dcosmic30, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dcosmic, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((5, 5), (5, 5), (5, 5))

In [180]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.579750899
496226.5426212941	96541.34153693542	269437.40309821675	-1928441.3798997358	1401221.6430270765
-19413664.00695389	-6055929.2249728255	-1928441.3798995449	80306511.07905662	-23831659.69196253
6641394.102308079	2108424.579750879	1401221.643026993	-23831659.691962913	13961632.845623164


In [181]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473726.474570977
647009.0811369446	138897.17270890815	310960.1564787712	-2499890.3801519116	1671688.8824865094
-22479456.449508905	-6988605.066998642	-2499890.3801519973	92043357.18543775	-28355007.711743876
7855159.798259621	2473726.4745712737	1671688.8824863958	-28355007.71174604	15915477.89555201


In [182]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

6405232.029924338	2077977.6826575808	697354.5570693449	-24059807.030419346	8319137.816911822
2077977.6826578246	718020.1282746888	149400.52635495152	-7482815.152958558	2607830.438294836
697354.5570693264	149400.52635493194	335729.1513240685	-2699977.708029737	1802649.6326798527
-24059807.03043552	-7482815.152965439	-2699977.7080230205	98334244.47419085	-30083566.183535084
8319137.816940438	2607830.438307992	1802649.6326699953	-30083566.1836229	17060637.332088664


In [183]:
np.shape(dsignal_dcosmic50)

(5, 19, 324)

In [184]:
dsignal_dcosmic50_ = np.concatenate([dsignal_dcosmic50, np.zeros_like(dsignal_dcosmic30)], axis=2)
dsignal_dcosmic30_ = np.concatenate([np.zeros_like(dsignal_dcosmic50), dsignal_dcosmic30], axis=2)

dsignal_dcosmic_multi = np.concatenate([dsignal_dcosmic50_, dsignal_dcosmic30_])

np.shape(dsignal_dcosmic50_), np.shape(dsignal_dcosmic30_), np.shape(dsignal_dcosmic_multi)

((5, 19, 648), (5, 19, 648), (10, 19, 648))

In [185]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dcosmic_multi, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 




 ... Success! ... 



In [186]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

9539077189.76009	4699062467.304879	-3793566785.955897	-30204876274.45651	4890748239.244772	-9536684220.731607	-4698204436.697295	3793554670.3935885	30196040435.182156	-4888259081.254623
4699062467.2954445	2681438429.277575	-2716175973.8133435	-12353821449.585	1265344060.3906302	-4698119894.350827	-2681073937.8858085	2716122523.3009934	12350510213.68771	-1264409034.188464
-3793566785.9022207	-2716175973.803107	3697388428.7369437	7403214754.726536	1751758662.677313	3793310549.048102	2716030687.8027267	-3697229754.035046	-7402351353.679704	-1751517185.1360176
-30204876274.314995	-12353821449.63124	7403214755.0802145	125862048670.84528	-20812945507.020832	30196415642.46067	12350944574.720057	-7403193159.663582	-125827016753.56006	20805518665.96384
4890748239.398234	1265344060.4087021	1751758662.6997545	-20812945508.125942	13043577015.263817	-4888830269.198083	-1264685233.1894965	-1751437914.217336	20806419502.35252	-13038689843.025333
-9536684220.724144	-4698119894.359445	3793310549.102787

In [187]:
np.linalg.det(FisherMatrixSKA_joint), np.linalg.det(FisherMatrixSKA_joint_)

(1.893120279696526e+29, 6.127975740830933e+74)

In [188]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in np.linalg.inv(FisherMatrixSKA_joint)]))

4.0294831661160583e-05	-8.699649604758185e-05	-3.567580567338526e-05	3.191531604555138e-06	3.0466319489029477e-06
-8.699649604738407e-05	0.00020202523659769512	8.671266478368432e-05	-6.088526158424483e-06	-8.357768311300435e-06
-3.567580567324954e-05	8.671266478373623e-05	4.532957723260507e-05	-2.3538798290344217e-06	-4.7985562475037825e-06
3.1915316045637484e-06	-6.088526158457173e-06	-2.353879829058179e-06	3.2095689589204496e-07	1.8907858722081115e-07
3.0466319488688754e-06	-8.357768311258237e-06	-4.798556247464513e-06	1.8907858721713672e-07	6.909803636360851e-07


## Cosmic Parameters + Galaxy Bias FISHER MATRIX

In [189]:
dsignal_dpars50x50 = np.concatenate([dsignal_dgbias50_, dsignal_dcosmic50])
dsignal_dpars30x70 = np.concatenate([dsignal_dgbias30_, dsignal_dcosmic30])
dsignal_dpars_joint = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dpars50x50), np.shape(dsignal_dpars30x70), np.shape(dsignal_dpars_joint)

((9, 19, 324), (9, 19, 324), (13, 19, 648))

In [190]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dpars50x50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dpars30x70, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dpars_joint, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((9, 9), (9, 9), (13, 13))

In [191]:
np.linalg.det(FisherMatrixSKA_50x50), np.linalg.det(FisherMatrixSKA_30x70), np.linalg.det(FisherMatrixSKA_joint)

(2.978620645887647e+53, 6.949421003610326e+53, 1.0781374427237098e+87)

In [192]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

25447240.777854692	7515169.987835224	-31713835.347808655	-9110094.934605015	2177814.361231719	1105856.1562454673	-932413.0557254107	-6618981.011715738	-2933471.9958783127
7515169.987835222	2808444.203160712	-9110094.934605015	-3225463.7597114304	1072997.0184735924	504090.3908093623	-332884.4190554553	-3396522.347946506	-474952.963605881
-31713835.347809885	-9110094.93460506	48443965.55355428	14405111.655087853	2321035.4568680176	305381.6409740199	1695881.1591249863	-10068274.409692686	11822762.336750662
-9110094.934605056	-3225463.7597113927	14405111.655087844	5400372.421185124	629545.2719306645	69738.45404022718	496145.3945584891	-2793044.5741037377	3003394.232552508
2177814.3612309718	1072997.0184736014	2321035.456869026	629545.2719306569	5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1105856.1562451809	504090.390809364	305381.6409744265	69738.45404022578	1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.57

In [193]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

33016651.384679236	8332488.267908776	-50360159.32634173	-12901331.246122913	1874549.3290753085	1015691.4324662106	-994719.3725617193	-5479385.917692468	-3961393.346895395
8332488.267908773	2733114.2927259943	-12901331.246122926	-4149358.7031630417	846945.2037477822	422633.8689456244	-337296.306303709	-2592730.776060814	-779727.1200034039
-50360159.32633923	-12901331.24612243	84500438.42434922	23254467.951169528	2067729.6980300494	117760.24214361416	1973312.5021055618	-9524773.356651565	13038348.119257785
-12901331.246122435	-4149358.7031629556	23254467.951169528	8118604.127277631	736809.8546538168	81090.78851551346	578643.8650978748	-3279873.2486417764	3469697.352632582
1874549.3290768762	846945.2037479944	2067729.6980268739	736809.8546534531	5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1015691.4324669703	422633.86894572224	117760.24214217364	81090.7885153587	1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473

In [194]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

3941526166.5513344	677901614.663729	4449560249.806032	752496017.8426421	-2276010839.323661	-392299191.9842474	-6420523876.82295	-1089686004.2573042	1863198.5876474048	876449.7025027502	-579137.503166111	-5951999.86552859	-1631735.9555959962
677901614.663729	152270188.05923846	752496017.8426422	164298276.02453616	-392299191.98424745	-88356498.31439497	-1089686004.2573042	-239362490.06637508	699265.0233474443	322475.68541597825	-197049.15242615773	-2241271.106895188	-319570.5071121953
4449560249.793868	752496017.841359	5303963291.056365	911027194.6856267	-2572330475.4796166	-437729693.71853423	-7526453920.805931	-1282385619.7756743	188397.07232881835	-130573.27160901953	607616.2824386378	-1432255.5442824054	5316793.274569787
752496017.8413591	164298276.02438536	911027194.6856267	203211054.58891585	-437729693.71853423	-96398372.37245576	-1282385619.7756743	-282613707.9482976	19575.470560387155	-63115.4232206587	212256.61568367854	-333837.510843183	1278889.661013332
-2276010839.3276496	-39

In [195]:
dsignal_dpars_joint_ = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic_multi])

np.shape(dsignal_dpars_joint_)

(18, 19, 648)

In [196]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dpars_joint_, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 




 ... Success! ... 



In [197]:
np.shape(FisherMatrixSKA_joint_), np.linalg.det(FisherMatrixSKA_joint_)

((18, 18), 1.9433768148766404e+130)

In [198]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

3941526166.5513344	677901614.663729	4449560249.806032	752496017.8426421	-2276010839.323661	-392299191.9842474	-6420523876.82295	-1089686004.2573042	5595667663.327287	3216391189.9399824	-3302383269.948417	-14609108519.024176	1282309891.233212	-5593804464.739638	-3215514740.2374783	3301804132.4452515	14603156519.158655	-1283941627.188808
677901614.663729	152270188.05923846	752496017.8426422	164298276.02453616	-392299191.98424745	-88356498.31439497	-1089686004.2573042	-239362490.06637508	931797468.6074542	535664462.89766383	-550028770.5021983	-2432102320.6241264	121116582.19955778	-931098203.5841068	-535341987.21224797	549831721.3497723	2429861049.517231	-121436152.70667002
4449560249.793868	752496017.841359	5303963291.056365	911027194.6856267	-2572330475.4796166	-437729693.71853423	-7526453920.805931	-1282385619.7756743	6462816938.950627	3713419866.118275	-3809378009.7195787	-16875199817.419388	1520370951.1752195	-6462628541.878301	-3713550439.3898854	3809985626.0020175	16873767561.87511

#  Joint Covariance Matrix 50x30 [all]

## Check Cauchy-Schwarz inequality 

The Cauchy-Schwarz inequality implies:

$$ \mathrm{COV}(X,Y) \leq \sqrt{\mathrm{VAR}(X)\cdot\mathrm{VAR}(Y)}$$

In [199]:
z_bins = np.array([0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])


In [200]:
CovMatrixSKA_joint = fm.cov_matrix_joint(d=dist[0], z_bins=z_bins, small_off_diagonal=True, rtol=1e-4, splits=[50,30], contributions='all')


 ... Loading Covariance Matrices (Joint-splittings Analysis) [all] [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 



In [201]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol = 1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [202]:
np.shape(CovMatrixSKA_50x50), np.shape(CovMatrixSKA_30x70), np.shape(CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 648, 648))

In [203]:
CovMatrixSKA_50x30 = np.array([CovMatrixSKA_joint[k][:324, 324:] for k,_ in enumerate(z_bins)])
CovMatrixSKA_30x50 = np.array([CovMatrixSKA_joint[k][324:, :324] for k,_ in enumerate(z_bins)])

In [204]:
np.shape(CovMatrixSKA_50x30), np.shape(CovMatrixSKA_30x50)

((19, 324, 324), (19, 324, 324))

In [205]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_50x30[n], CovMatrixSKA_30x50[n].T, rtol=1e-10), np.allclose(CovMatrixSKA_30x50[n], CovMatrixSKA_50x30[n].T, rtol = 1e-10))

0 True True
1 True True
2 True True
3 True True
4 True True
5 True True
6 True True
7 True True
8 True True
9 True True
10 True True
11 True True
12 True True
13 True True
14 True True
15 True True
16 True True
17 True True
18 True True


In [206]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol=1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [207]:
np.shape(CovMatrixSKA_50x30), np.shape(CovMatrixSKA_30x50)

((19, 324, 324), (19, 324, 324))

In [208]:
CovMatrixSKA_50x30[0,0,0], CovMatrixSKA_30x50[0,0,0]

(1.761081877983889e-05, 1.761081877983889e-05)

In [209]:
CovMatrixSKA_50x50[0,0,0], CovMatrixSKA_30x70[0,0,0]

(1.1406613424594502e-05, 2.728761334220845e-05)

In [210]:
CovMatrixSKA_50x30[0,17,267], CovMatrixSKA_30x50[0,267,17]

(-4.188698962835081e-09, -4.188698962835081e-09)

In [211]:
CovMatrixSKA_50x50[0,17,267], CovMatrixSKA_30x70[0,267,17]

(-4.188698962835081e-09, -5.405294849542528e-09)

## Checking directly from Joint Cov Matrix 50x30

Compare off diagonal elements (covariances) versus the correspondent diagonal ones (variances).

In [212]:
np.shape(CovMatrixSKA_joint), type(CovMatrixSKA_joint)

((19, 648, 648), list)

In [213]:
36*9

324

In [214]:
print(' Monopole BB (d=d), 50x50 = ', CovMatrixSKA_joint[0][0,0],'\n','Monopole BB (d=d), 30x70 = ', CovMatrixSKA_joint[0][324,324] ) 

 Monopole BB (d=d), 50x50 =  1.1406613424594502e-05 
 Monopole BB (d=d), 30x70 =  2.728761334220845e-05


In [215]:
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][0,324],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][324,0] ) 

 Monopole BB (d=d), 50x30 =  1.761081877983889e-05 
 Monopole BB (d=d), 30x50 =  1.761081877983889e-05


In [216]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][0,324], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][324,0])

0.15 True True
0.25 True True
0.35 True True
0.45 True True
0.55 True True
0.65 True True
0.75 True True
0.85 True True
0.95 True True
1.05 True True
1.15 True True
1.25 True True
1.35 True True
1.45 True True
1.55 True True
1.65 True True
1.75 True True
1.85 True True
1.95 True True


In [217]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]), CovMatrixSKA_joint[k][0,324])

0.15 1.764254110592856e-05 1.761081877983889e-05
0.25 7.014350303522421e-06 7.000424397731244e-06
0.35 3.904079159778656e-06 3.895101127116166e-06
0.45 2.5784765980908824e-06 2.57114829840431e-06
0.55 1.8931986511136867e-06 1.8860669201036704e-06
0.65 1.4969289169443364e-06 1.4890758213620874e-06
0.75 1.2532721988093958e-06 1.2437159931673294e-06
0.85 1.1007803772214066e-06 1.0882357772332196e-06
0.95 1.0104386161984314e-06 9.92815265805662e-07
1.05 9.703620669081107e-07 9.439141037718107e-07
1.15 9.797925594098938e-07 9.378224104633536e-07
1.25 1.0534917202039827e-06 9.825635624950322e-07
1.35 1.2326007500359773e-06 1.1041080122051723e-06
1.45 1.6096843827924573e-06 1.361464441548065e-06
1.55 2.436372271029366e-06 1.9147195310493815e-06
1.65 4.3288471808299055e-06 3.1492163021338824e-06
1.75 8.978984214121797e-06 6.1106398851286035e-06
1.85 2.1513371013254955e-05 1.3937722616731117e-05
1.95 5.784949838880617e-05 3.630171649833516e-05


In [218]:
print(' MonoBQuadBF (d=d), 50x50 = ', CovMatrixSKA_joint[0][179,179],'\n','MonoBQuadBF (d=d) (d=d), 30x70 = ', CovMatrixSKA_joint[0][179+324,179+324] )
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][179,179+324],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][179+324,179] )  

 MonoBQuadBF (d=d), 50x50 =  1.9425535215326826e-06 
 MonoBQuadBF (d=d) (d=d), 30x70 =  4.3165840334060815e-06
 Monopole BB (d=d), 50x30 =  2.891194497518877e-06 
 Monopole BB (d=d), 30x50 =  2.891194497518877e-06


In [219]:
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][179,179+324], 
          np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][324+179,179])

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [220]:
len(CovMatrixSKA_50x50[0])

324

In [221]:
i_element = 288
j_element = i_element
l = len(CovMatrixSKA_50x50[0])

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  1.2444286340731441e-08 
 Element (d=d), 30x70 =  1.563436441154079e-08
 Element (d=d), 50x30 =  1.0270945457089037e-08 
 Element (d=d), 30x50 =  1.0270945457089037e-08 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [222]:
i_element = 56
j_element = i_element
l = len(CovMatrixSKA_50x50[0])

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  6.551780712795955e-08 
 Element (d=d), 30x70 =  1.6800215155336124e-07
 Element (d=d), 50x30 =  1.0388525556579684e-07 
 Element (d=d), 30x50 =  1.0388525556579684e-07 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [223]:
print(CovMatrixSKA_50x50[0][24,24], CovMatrixSKA_50x50[0][36,36])
print(CovMatrixSKA_50x50[0][24,36], CovMatrixSKA_50x50[0][36,24])

5.093876193886468e-07 1.0577598333198072e-06
5.353246240816097e-08 5.353246240816097e-08


In [224]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_50x50[k][10,25]**2 <= CovMatrixSKA_50x50[k][10,10]*CovMatrixSKA_50x50[k][25,25])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


In [225]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_30x70[k][18,30]**2 <= CovMatrixSKA_30x70[k][18,18]*CovMatrixSKA_30x70[k][30,30])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


## Signal-to-noise

In [226]:
splits = [2., 10/3.]

mySignals = [Signal(solver=Solver(zs=[0], **params_dict0), n_split=m) for m in splits]


In [227]:
d = np.linspace(20,160,36)
z_bins = np.linspace(0.15, 1.95, 19)

monopole_sigs = [mySignals[k]._monopole(d=d, z=z_bins) for k,_ in enumerate(splits)]

np.shape(monopole_sigs)

(2, 3, 19, 36)

In [228]:
monopoleBB_signal = [monopole_sigs[k][0] for k,_ in enumerate(splits)]

In [229]:
VarMONO_50x50 = [CovMatrixSKA_50x50[k,:36,:36] for k,_ in  enumerate(z_bins)]
VarMONO_30x70 = [CovMatrixSKA_30x70[k,:36,:36] for k,_ in  enumerate(z_bins)]
#VarMONO_10x90 = [CovMatrixSKA_10x90[k,:36,:36] for k,_ in  enumerate(z_bins)]

In [230]:

#covs = [ VarMONO_50x50, VarMONO_30x70, VarMONO_10x90 ]

covs = [ VarMONO_50x50, VarMONO_30x70 ]


cumSNR, SNRz = fm.signal_to_noise(monopoleBB_signal[0], covs[0])


 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 58.77421213378112
Bin 2: 89.48345521707645
Bin 3: 114.76519820946098
Bin 4: 134.716436740866
Bin 5: 149.53509304639073
Bin 6: 159.5306944397853
Bin 7: 164.89350644693744
Bin 8: 165.8820114819602
Bin 9: 162.66901920794453
Bin 10: 155.44381004466567
Bin 11: 144.78901213682857
Bin 12: 131.1870337188347
Bin 13: 115.31104126390467
Bin 14: 98.24887356301002
Bin 15: 80.54138755170176
Bin 16: 63.4308923544652
Bin 17: 47.776283841079085
Bin 18: 34.10909687634405
Bin 19: 23.019044873900647

--------------------------------------------------

 Cumulative SNR : 521.0877057700761 

--------------------------------------------------


In [231]:
for k,_ in enumerate(splits):
    cumSNR, SNRz = fm.signal_to_noise(monopoleBB_signal[k], covs[k])


 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 58.77421213378112
Bin 2: 89.48345521707645
Bin 3: 114.76519820946098
Bin 4: 134.716436740866
Bin 5: 149.53509304639073
Bin 6: 159.5306944397853
Bin 7: 164.89350644693744
Bin 8: 165.8820114819602
Bin 9: 162.66901920794453
Bin 10: 155.44381004466567
Bin 11: 144.78901213682857
Bin 12: 131.1870337188347
Bin 13: 115.31104126390467
Bin 14: 98.24887356301002
Bin 15: 80.54138755170176
Bin 16: 63.4308923544652
Bin 17: 47.776283841079085
Bin 18: 34.10909687634405
Bin 19: 23.019044873900647

--------------------------------------------------

 Cumulative SNR : 521.0877057700761 

--------------------------------------------------

 ... Computing the S/N ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 

--------------------------------------------------

 Individual SNRs: 

Bin 1: 50.28

## Test Fisher Matrix

### Load the derivatives

In [232]:
with open('derivatives.pkl', 'rb') as File:
    derivatives_dict = pk.load(File)
    
derivatives_50x50 = derivatives_dict['split_50x50']
derivatives_30x70 = derivatives_dict['split_30x70']

In [233]:
dsignal_dsbias50 = derivatives_50x50['dsignal_dsbias']

dsignal_dgbias50_ = derivatives_50x50['dsignal_dgbias']

dsignal_dcosmic50 = derivatives_50x50['dsignal_dcosmic']

dsignal_dnum50 = derivatives_50x50['dsignal_dnumev']

np.shape(dsignal_dsbias50), np.shape(dsignal_dgbias50_), np.shape(dsignal_dcosmic50), np.shape(dsignal_dnum50)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [234]:
dsignal_dsbias30 = derivatives_30x70['dsignal_dsbias']

dsignal_dgbias30_ = derivatives_30x70['dsignal_dgbias']

dsignal_dcosmic30 = derivatives_30x70['dsignal_dcosmic']

dsignal_dnum30 = derivatives_30x70['dsignal_dnumev']

np.shape(dsignal_dsbias30), np.shape(dsignal_dgbias30_), np.shape(dsignal_dcosmic30), np.shape(dsignal_dnum30)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [235]:
dsignal_dtheta50 = np.concatenate([dsignal_dsbias50, dsignal_dnum50, dsignal_dgbias50_, dsignal_dcosmic50])

np.shape(dsignal_dtheta50)

(21, 19, 324)

In [236]:
dsignal_dtheta30 = np.concatenate([dsignal_dsbias30, dsignal_dnum30, dsignal_dgbias30_, dsignal_dcosmic30])

np.shape(dsignal_dtheta30)

(21, 19, 324)

In [237]:
dsignal_dsB50 = np.concatenate([dsignal_dsbias50[:4], np.zeros_like(dsignal_dsbias50[:4])], axis=2)
dsignal_dsB30 = np.concatenate([np.zeros_like(dsignal_dsbias30[:4]), dsignal_dsbias30[:4]], axis=2)

dsignal_dsM = np.concatenate([dsignal_dsbias50[4:], dsignal_dsbias30[4:]], axis=2)
dsignal_dnum = np.concatenate([dsignal_dnum50, dsignal_dnum30], axis=2)
dsignal_dgbias50 = np.concatenate([dsignal_dgbias50_, np.zeros_like(dsignal_dgbias50_[:4])], axis=2)
dsignal_dgbias30 = np.concatenate([np.zeros_like(dsignal_dgbias30_[:4]), dsignal_dgbias30_], axis=2)
dsignal_dcosmic = np.concatenate([dsignal_dcosmic50, dsignal_dcosmic30], axis=2)

In [238]:
dsignal_dtheta_multi = np.concatenate([dsignal_dsB50, dsignal_dsB30, dsignal_dsM, dsignal_dnum, dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dtheta_multi)

(29, 19, 648)

## Cosmic Parameters Fisher Matrices

In [239]:
Inv_CovMatrixSKA_joint = np.array([np.linalg.inv(CovMatrixSKA_joint[i]) for i,k in enumerate(z_bins)])

np.shape(Inv_CovMatrixSKA_50x50), np.shape(Inv_CovMatrixSKA_30x70), np.shape(Inv_CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 648, 648))

In [240]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(Inv_CovMatrixSKA_50x50[n], Inv_CovMatrixSKA_50x50[n].T, rtol=1e-4), 
          np.allclose(Inv_CovMatrixSKA_30x70[n], Inv_CovMatrixSKA_30x70[n].T, rtol=1e-4),
          np.allclose(Inv_CovMatrixSKA_joint[n], Inv_CovMatrixSKA_joint[n].T, rtol=1e-1))

0 True True False
1 True True False
2 True True False
3 True True False
4 True True False
5 True True False
6 True True False
7 True True False
8 True True False
9 True True False
10 True True False
11 True True False
12 True True False
13 True True False
14 True True False
15 True True False
16 True True False
17 True True False
18 True True False


In [241]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dcosmic50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dcosmic30, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dcosmic, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((5, 5), (5, 5), (5, 5))

In [242]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.579750899
496226.5426212941	96541.34153693542	269437.40309821675	-1928441.3798997358	1401221.6430270765
-19413664.00695389	-6055929.2249728255	-1928441.3798995449	80306511.07905662	-23831659.69196253
6641394.102308079	2108424.579750879	1401221.643026993	-23831659.691962913	13961632.845623164


In [243]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473726.474570977
647009.0811369446	138897.17270890815	310960.1564787712	-2499890.3801519116	1671688.8824865094
-22479456.449508905	-6988605.066998642	-2499890.3801519973	92043357.18543775	-28355007.711743876
7855159.798259621	2473726.4745712737	1671688.8824863958	-28355007.71174604	15915477.89555201


In [244]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

23540005.405536592	6309151.987861257	7374443.615792481	-91382209.91576387	42721937.60725491
5687407.206411818	1171329.3425980997	2847530.417657021	-23581595.63987485	12834068.855523234
5415085.5710737845	1575172.0305968022	1417495.466707767	-19559345.331203572	9253628.711412916
-90211434.07125682	-25681821.13135975	-24965943.863310527	338838413.19141704	-157779418.11353573
35063075.88393482	9294585.562959393	11656586.058249475	-138047395.64778018	65076615.114773355


In [245]:
np.shape(dsignal_dcosmic50)

(5, 19, 324)

In [246]:
dsignal_dcosmic50_ = np.concatenate([dsignal_dcosmic50, np.zeros_like(dsignal_dcosmic30)], axis=2)
dsignal_dcosmic30_ = np.concatenate([np.zeros_like(dsignal_dcosmic50), dsignal_dcosmic30], axis=2)

dsignal_dcosmic_multi = np.concatenate([dsignal_dcosmic50_, dsignal_dcosmic30_])

np.shape(dsignal_dcosmic50_), np.shape(dsignal_dcosmic30_), np.shape(dsignal_dcosmic_multi)

((5, 19, 648), (5, 19, 648), (10, 19, 648))

In [247]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dcosmic_multi, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



In [248]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

3.932845848544591e+21	9.877624561515537e+20	1.1256532326968956e+21	-1.5985070100273877e+22	7.352933399336688e+21	-3.9328349577216425e+21	-9.877594618408829e+20	-1.1256497525316393e+21	1.5985029064266695e+22	-7.35290287841733e+21
1.296054332118401e+21	2.406577040183968e+20	5.8468488620672877e+20	-5.468440150375117e+21	2.850616129422429e+21	-1.2960551977292385e+21	-2.4065794648655426e+20	-5.846851636231961e+20	5.468443262599428e+21	-2.850614479115544e+21
4.764190753606881e+20	2.8530083172578994e+20	-1.1596996605700006e+20	-9.934899754744026e+20	1.1759439414377762e+21	-4.7639738305237405e+20	-2.85294810398686e+20	1.1597673381697323e+20	9.934088171171291e+20	-1.1758999423775124e+21
-1.507067585650983e+22	-4.6150450950975e+21	-1.3406363190187802e+21	6.2189990778704805e+22	-1.7563626327711527e+22	1.5070609003316907e+22	4.615026739017164e+21	1.3406147355893508e+21	-6.218973999090837e+22	1.7563448729482666e+22
7.028917694706071e+21	2.0681126378989148e+21	2.0733841650135678e+21	-2.5254695151545

In [249]:
np.linalg.det(FisherMatrixSKA_joint), np.linalg.det(FisherMatrixSKA_joint_)

(5.561916506041953e+30, 9.66869206281804e+136)

In [250]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in np.linalg.inv(FisherMatrixSKA_joint)]))

4.7474308679761215e-06	-4.5069396456612225e-07	8.771220924825273e-06	1.1142354286531753e-06	-1.573490139146783e-06
-1.094285122975047e-05	-1.9822679027962325e-06	3.776912596878033e-06	-3.1781196436359603e-07	6.267167335983491e-06
-3.0470006230043076e-06	-2.843749562065036e-06	1.5876405054192338e-05	1.6761752027830387e-06	4.3675018775844595e-06
7.099011238127303e-08	1.908640427424329e-07	1.2502717509652293e-06	6.526863610212872e-08	-1.0378383314982536e-07
-2.986116683095764e-07	1.4402066036729706e-06	-5.456933251494058e-06	-7.167383564437936e-07	-1.0344202467159471e-06


## Cosmic Parameters + Galaxy Bias FISHER MATRIX

In [251]:
dsignal_dpars50x50 = np.concatenate([dsignal_dgbias50_, dsignal_dcosmic50])
dsignal_dpars30x70 = np.concatenate([dsignal_dgbias30_, dsignal_dcosmic30])
dsignal_dpars_joint = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dpars50x50), np.shape(dsignal_dpars30x70), np.shape(dsignal_dpars_joint)

((9, 19, 324), (9, 19, 324), (13, 19, 648))

In [252]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dpars50x50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dpars30x70, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dpars_joint, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 




 ... Success! ... 



((9, 9), (9, 9), (13, 13))

In [253]:
np.linalg.det(FisherMatrixSKA_50x50), np.linalg.det(FisherMatrixSKA_30x70), np.linalg.det(FisherMatrixSKA_joint)

(2.978620645887647e+53, 6.949421003610326e+53, -1.1009360003469574e+138)

In [254]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

25447240.777854692	7515169.987835224	-31713835.347808655	-9110094.934605015	2177814.361231719	1105856.1562454673	-932413.0557254107	-6618981.011715738	-2933471.9958783127
7515169.987835222	2808444.203160712	-9110094.934605015	-3225463.7597114304	1072997.0184735924	504090.3908093623	-332884.4190554553	-3396522.347946506	-474952.963605881
-31713835.347809885	-9110094.93460506	48443965.55355428	14405111.655087853	2321035.4568680176	305381.6409740199	1695881.1591249863	-10068274.409692686	11822762.336750662
-9110094.934605056	-3225463.7597113927	14405111.655087844	5400372.421185124	629545.2719306645	69738.45404022718	496145.3945584891	-2793044.5741037377	3003394.232552508
2177814.3612309718	1072997.0184736014	2321035.456869026	629545.2719306569	5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1105856.1562451809	504090.390809364	305381.6409744265	69738.45404022578	1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.57

In [255]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

33016651.384679236	8332488.267908776	-50360159.32634173	-12901331.246122913	1874549.3290753085	1015691.4324662106	-994719.3725617193	-5479385.917692468	-3961393.346895395
8332488.267908773	2733114.2927259943	-12901331.246122926	-4149358.7031630417	846945.2037477822	422633.8689456244	-337296.306303709	-2592730.776060814	-779727.1200034039
-50360159.32633923	-12901331.24612243	84500438.42434922	23254467.951169528	2067729.6980300494	117760.24214361416	1973312.5021055618	-9524773.356651565	13038348.119257785
-12901331.246122435	-4149358.7031629556	23254467.951169528	8118604.127277631	736809.8546538168	81090.78851551346	578643.8650978748	-3279873.2486417764	3469697.352632582
1874549.3290768762	846945.2037479944	2067729.6980268739	736809.8546534531	5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1015691.4324669703	422633.86894572224	117760.24214217364	81090.7885153587	1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473

In [256]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

2.5385057226077438e+20	3.227893879290389e+20	2.7019845864174243e+20	3.502117430532662e+20	-1.569666143420626e+20	-1.973090600528217e+20	-3.674131951524024e+20	-4.818137779832439e+20	-1669004975901248.2	-466275583996496.8	-527935366119441.25	6094426025613464.0	535279544340240.0
3.2278938792903967e+20	3.280481580199732e+20	3.502117430532675e+20	3.5781189760977496e+20	-1.9730906005282164e+20	-1.998848468028931e+20	-4.8181377798324316e+20	-4.93806103295406e+20	-2019307803190054.0	-561719742951867.9	-627520243199851.6	7539001999290952.0	-3994270677557522.0
2.7019847210432994e+20	3.502117465268752e+20	2.8706340177849935e+20	3.798077826821554e+20	-1.6725915939822993e+20	-2.1412374801571945e+20	-3.898952347215402e+20	-5.224052637520908e+20	-1961909434001332.0	-547963540457907.0	-620519337760174.0	7168986881233536.0	493155245051472.0
3.5021174652687555e+20	3.578118987555899e+20	3.7980778268215575e+20	3.902142832792029e+20	-2.1412374801571997e+20	-2.1804086733745786e+20	-5.2240526375209206e+20	-

In [257]:
dsignal_dpars_joint_ = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic_multi])

np.shape(dsignal_dpars_joint_)

(18, 19, 648)

In [258]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dpars_joint_, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 




 ... Success! ... 



In [259]:
np.shape(FisherMatrixSKA_joint_), np.linalg.det(FisherMatrixSKA_joint_)

((18, 18), 1.504031017503593e+241)

In [260]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

2.5385057226077438e+20	3.227893879290389e+20	2.7019845864174243e+20	3.502117430532662e+20	-1.569666143420626e+20	-1.973090600528217e+20	-3.674131951524024e+20	-4.818137779832439e+20	8.507161990194598e+20	6.7433081472865894e+19	4.521124541886201e+20	-4.1889112535932737e+21	1.2328145716002357e+21	-8.507178680244354e+20	-6.7433547748449755e+19	-4.5211298212398616e+20	4.188917348019299e+21	-1.2328140363206917e+21
3.2278938792903967e+20	3.280481580199732e+20	3.502117430532675e+20	3.5781189760977496e+20	-1.9730906005282164e+20	-1.998848468028931e+20	-4.8181377798324316e+20	-4.93806103295406e+20	7.12442073025591e+20	1.2952273623431587e+20	1.898281535889699e+20	-3.284621651668759e+21	5.674740900597645e+20	-7.124440923333941e+20	-1.2952329795405881e+20	-1.8982878110921336e+20	3.2846291906707577e+21	-5.674780843304423e+20
2.7019847210432994e+20	3.502117465268752e+20	2.8706340177849935e+20	3.798077826821554e+20	-1.6725915939822993e+20	-2.1412374801571945e+20	-3.898952347215402e+20	-5.224052637520

#  Joint Covariance Matrix 50x30 [all] REMOVE ONE HEXADECAPOLE

## Check Cauchy-Schwarz inequality 

The Cauchy-Schwarz inequality implies:

$$ \mathrm{COV}(X,Y) \leq \sqrt{\mathrm{VAR}(X)\cdot\mathrm{VAR}(Y)}$$

In [395]:
z_bins = np.array([0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])

n_sep = len(dist[0])

ind_to_remove = list(range(7*n_sep,8*n_sep))


In [396]:
CovMatrixSKA_joint_ = fm.cov_matrix_joint(d=dist[0], z_bins=z_bins, small_off_diagonal=True, rtol=1e-4, splits=[50,30], contributions='all')


 ... Loading Covariance Matrices (Joint-splittings Analysis) [all] [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 



In [397]:
print('\n ... Remove the HEXADECAPOLE from 50x50 split ... \n')

CovMatrixSKA_joint = np.delete(CovMatrixSKA_joint_, ind_to_remove, axis=1)
CovMatrixSKA_joint = np.delete(CovMatrixSKA_joint, ind_to_remove, axis=2)

np.shape(CovMatrixSKA_joint_), np.shape(CovMatrixSKA_joint)


 ... Remove the HEXADECAPOLE from 50x50 split ... 



((19, 648, 648), (19, 612, 612))

In [398]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol = 1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [399]:
np.shape(CovMatrixSKA_50x50), np.shape(CovMatrixSKA_30x70), np.shape(CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 612, 612))

## Checking directly from Joint Cov Matrix 50x30

Compare off diagonal elements (covariances) versus the correspondent diagonal ones (variances).

In [400]:
np.shape(CovMatrixSKA_joint), type(CovMatrixSKA_joint)

((19, 612, 612), numpy.ndarray)

In [401]:
2*36*9-36

612

In [402]:
print(' Monopole BB (d=d), 50x50 = ', CovMatrixSKA_joint[0][0,0],'\n','Monopole BB (d=d), 30x70 = ', CovMatrixSKA_joint[0][324,324] ) 

 Monopole BB (d=d), 50x50 =  1.1406613424594502e-05 
 Monopole BB (d=d), 30x70 =  2.710848522234923e-06


In [403]:
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][0,324],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][324,0] ) 

 Monopole BB (d=d), 50x30 =  5.4529116478367385e-06 
 Monopole BB (d=d), 30x50 =  5.4529116478367385e-06


In [404]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][0,324], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][324,0])

0.15 True True
0.25 True True
0.35 True True
0.45 True True
0.55 True True
0.65 True True
0.75 True True
0.85 True True
0.95 True True
1.05 True True
1.15 True True
1.25 True True
1.35 True True
1.45 True True
1.55 True True
1.65 True True
1.75 True True
1.85 True True
1.95 True True


In [405]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]), CovMatrixSKA_joint[k][0,324])

0.15 5.560719480945522e-06 5.4529116478367385e-06
0.25 2.4072793607924924e-06 2.369059867465096e-06
0.35 1.4413122011281243e-06 1.4220736106850454e-06
0.45 1.01448263246394e-06 1.0026263272710078e-06
0.55 7.879982613735045e-07 7.794185195347171e-07
0.65 6.55217457542722e-07 6.480221578160664e-07
0.75 5.739319958027031e-07 5.669247611993325e-07
0.85 5.249111351607153e-07 5.170558619524636e-07
0.95 4.992546805844021e-07 4.892352109090447e-07
1.05 4.938993000620163e-07 4.795519510605689e-07
1.15 5.099649237689384e-07 4.874310612797096e-07
1.25 5.55173644388643e-07 5.165167960467193e-07
1.35 6.491797350855918e-07 5.768559035990331e-07
1.45 8.354725105188151e-07 6.897462742997723e-07
1.55 1.2312485858385983e-06 9.101841100116461e-07
1.65 2.1237123874642116e-06 1.3640533571886951e-06
1.75 4.3000643149346946e-06 2.380706734573997e-06
1.85 1.014186658029927e-05 4.920628597744649e-06
1.95 2.7036451944849437e-05 1.1874110612053319e-05


In [406]:
print(' MonoBQuadBF (d=d), 50x50 = ', CovMatrixSKA_joint[0][179,179],'\n','MonoBQuadBF (d=d) (d=d), 30x70 = ', CovMatrixSKA_joint[0][179+324,179+324] )
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][179,179+324],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][179+324,179] )  

 MonoBQuadBF (d=d), 50x50 =  1.9425535215326826e-06 
 MonoBQuadBF (d=d) (d=d), 30x70 =  5.802107326787381e-07
 Monopole BB (d=d), 50x30 =  1.0478145760334263e-06 
 Monopole BB (d=d), 30x50 =  1.0478145760334263e-06


In [407]:
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][179,179+324], 
          np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][324+179,179])

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [408]:
len(CovMatrixSKA_30x50[0])

324

In [409]:
i_element = 288
j_element = i_element
l = int((2*36*9-36)/2)

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  2.728761334220845e-05 
 Element (d=d), 30x70 =  5.164601290296824e-09
 Element (d=d), 50x30 =  0.0 
 Element (d=d), 30x50 =  0.0 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [410]:
i_element = 56
j_element = i_element
l = len(CovMatrixSKA_50x50[0])

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  6.551780712795955e-08 
 Element (d=d), 30x70 =  2.037598373627894e-08
 Element (d=d), 50x30 =  3.645579650706823e-08 
 Element (d=d), 30x50 =  3.645579650706823e-08 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [411]:
print(CovMatrixSKA_50x50[0][24,24], CovMatrixSKA_50x50[0][36,36])
print(CovMatrixSKA_50x50[0][24,36], CovMatrixSKA_50x50[0][36,24])

5.093876193886468e-07 1.0577598333198072e-06
5.353246240816097e-08 5.353246240816097e-08


In [412]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_50x50[k][10,25]**2 <= CovMatrixSKA_50x50[k][10,10]*CovMatrixSKA_50x50[k][25,25])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


In [413]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_30x70[k][18,30]**2 <= CovMatrixSKA_30x70[k][18,18]*CovMatrixSKA_30x70[k][30,30])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


## Test Fisher Matrix

### Load the derivatives

In [414]:
with open('derivatives.pkl', 'rb') as File:
    derivatives_dict = pk.load(File)
    
derivatives_50x50 = derivatives_dict['split_50x50']
derivatives_30x70 = derivatives_dict['split_30x70']

In [415]:
dsignal_dsbias50 = derivatives_50x50['dsignal_dsbias']

dsignal_dgbias50_ = derivatives_50x50['dsignal_dgbias']

dsignal_dcosmic50 = derivatives_50x50['dsignal_dcosmic']

dsignal_dnum50 = derivatives_50x50['dsignal_dnumev']

np.shape(dsignal_dsbias50), np.shape(dsignal_dgbias50_), np.shape(dsignal_dcosmic50), np.shape(dsignal_dnum50)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [416]:
dsignal_dsbias30 = derivatives_30x70['dsignal_dsbias']

dsignal_dgbias30_ = derivatives_30x70['dsignal_dgbias']

dsignal_dcosmic30 = derivatives_30x70['dsignal_dcosmic']

dsignal_dnum30 = derivatives_30x70['dsignal_dnumev']

np.shape(dsignal_dsbias30), np.shape(dsignal_dgbias30_), np.shape(dsignal_dcosmic30), np.shape(dsignal_dnum30)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [626]:
dsignal_dtheta50 = np.concatenate([dsignal_dsbias50, dsignal_dnum50, dsignal_dgbias50_, dsignal_dcosmic50])

np.shape(dsignal_dtheta50)

(21, 19, 324)

In [417]:
dsignal_dtheta30 = np.concatenate([dsignal_dsbias30, dsignal_dnum30, dsignal_dgbias30_, dsignal_dcosmic30])

np.shape(dsignal_dtheta30)

(21, 19, 324)

In [418]:
dsignal_dsB50 = np.concatenate([dsignal_dsbias50[:4], np.zeros_like(dsignal_dsbias50[:4])], axis=2)
dsignal_dsB30 = np.concatenate([np.zeros_like(dsignal_dsbias30[:4]), dsignal_dsbias30[:4]], axis=2)

dsignal_dsM = np.concatenate([dsignal_dsbias50[4:], dsignal_dsbias30[4:]], axis=2)
dsignal_dnum = np.concatenate([dsignal_dnum50, dsignal_dnum30], axis=2)
dsignal_dgbias50 = np.concatenate([dsignal_dgbias50_, np.zeros_like(dsignal_dgbias50_[:4])], axis=2)
dsignal_dgbias30 = np.concatenate([np.zeros_like(dsignal_dgbias30_[:4]), dsignal_dgbias30_], axis=2)
dsignal_dcosmic = np.concatenate([dsignal_dcosmic50, dsignal_dcosmic30], axis=2)

In [419]:
dsignal_dtheta_multi = np.concatenate([dsignal_dsB50, dsignal_dsB30, dsignal_dsM, dsignal_dnum, dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

dsignal_dtheta_multi = np.delete(dsignal_dtheta_multi, ind_to_remove, axis=2)
dsignal_dcosmic = np.delete(dsignal_dcosmic, ind_to_remove, axis=2)

np.shape(dsignal_dtheta_multi), np.shape(dsignal_dcosmic)

((29, 19, 612), (5, 19, 612))

## Cosmic Parameters Fisher Matrices

In [421]:
Inv_CovMatrixSKA_joint = np.array([np.linalg.pinv(CovMatrixSKA_joint[i]) for i,k in enumerate(z_bins)])

np.shape(Inv_CovMatrixSKA_50x50), np.shape(Inv_CovMatrixSKA_30x70), np.shape(Inv_CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 612, 612))

In [424]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(Inv_CovMatrixSKA_50x50[n], Inv_CovMatrixSKA_50x50[n].T, rtol=1e-4), 
          np.allclose(Inv_CovMatrixSKA_30x70[n], Inv_CovMatrixSKA_30x70[n].T, rtol=1e-4),
          np.allclose(Inv_CovMatrixSKA_joint[n], Inv_CovMatrixSKA_joint[n].T, rtol=1e-1))

0 True True False
1 True True False
2 True True False
3 True True False
4 True True False
5 True True False
6 True True False
7 True True False
8 True True False
9 True True False
10 True True False
11 True True False
12 True True False
13 True True False
14 True True False
15 True True False
16 True True False
17 True True False
18 True True False


In [425]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dcosmic50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dcosmic30, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dcosmic, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((5, 5), (5, 5), (5, 5))

In [426]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.579750899
496226.5426212941	96541.34153693542	269437.40309821675	-1928441.3798997358	1401221.6430270765
-19413664.00695389	-6055929.2249728255	-1928441.3798995449	80306511.07905662	-23831659.69196253
6641394.102308079	2108424.579750879	1401221.643026993	-23831659.691962913	13961632.845623164


In [427]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473726.474570977
647009.0811369446	138897.17270890815	310960.1564787712	-2499890.3801519116	1671688.8824865094
-22479456.449508905	-6988605.066998642	-2499890.3801519973	92043357.18543775	-28355007.711743876
7855159.798259621	2473726.4745712737	1671688.8824863958	-28355007.71174604	15915477.89555201


In [428]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

6251295.368837502	2026141.1179901503	687527.5698482153	-23482439.4055779	8178810.243777531
2026141.117990104	699264.2519198089	148489.37206981913	-7297374.249152005	2565456.666756784
687527.5698483501	148489.37206987408	327476.46734001266	-2658934.1807957934	1765010.6975681668
-23482439.405577064	-7297374.249151797	-2658934.180795723	95961186.90261878	-29573082.66786928
8178810.243777963	2565456.666756904	1765010.6975683053	-29573082.667871144	16756273.07661894


In [429]:
np.shape(dsignal_dcosmic50)

(5, 19, 324)

In [430]:
dsignal_dcosmic50_ = np.concatenate([dsignal_dcosmic50, np.zeros_like(dsignal_dcosmic30)], axis=2)
dsignal_dcosmic30_ = np.concatenate([np.zeros_like(dsignal_dcosmic50), dsignal_dcosmic30], axis=2)

dsignal_dcosmic_multi = np.concatenate([dsignal_dcosmic50_, dsignal_dcosmic30_])

dsignal_dcosmic_multi = np.delete(dsignal_dcosmic_multi, ind_to_remove, axis=2)

np.shape(dsignal_dcosmic50_), np.shape(dsignal_dcosmic30_), np.shape(dsignal_dcosmic_multi)

((5, 19, 648), (5, 19, 648), (10, 19, 612))

In [431]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dcosmic_multi, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



In [432]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

703872218.150775	319797474.72804046	-212245379.1844078	-2379823127.2053742	458883058.24678373	-700743671.1921499	-318511365.99167585	211724126.9584626	2368813880.4830008	-459591094.76312566
319797474.01400304	159990530.22853753	-124860282.52903669	-966871471.0093046	200114130.32489592	-318467386.31754535	-159384846.35101536	124514636.82020631	962572701.8048247	-200779356.09356847
-212245376.77242467	-124860281.60211782	136516642.24279994	589854186.6564937	-45670267.071075976	211615418.26900944	124472377.89075033	-136052453.63362306	-588064860.4949622	47237823.61807507
-2379823127.2024417	-966871472.6118709	589854192.4733844	10011341184.872478	-1454169894.2982576	2369131650.6487646	962834030.5090423	-588401478.1503981	-9967115738.710846	1456780753.308046
458883063.35614103	200114133.26483417	-45670270.49451372	-1454169908.1728764	789790775.2420641	-456879208.47708875	-199394060.95055488	45932501.20271988	1447408663.2480824	-786255353.2113926
-700743671.1870086	-318467387.0287613	2116154

In [433]:
np.linalg.det(FisherMatrixSKA_joint), np.linalg.det(FisherMatrixSKA_joint_)

(1.6519686050731574e+29, 7.699403863968864e+68)

In [434]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in np.linalg.inv(FisherMatrixSKA_joint)]))

4.1401384047626375e-05	-8.945846139545268e-05	-3.661641439888493e-05	3.2792061676656403e-06	3.1326978374865317e-06
-8.945846139546193e-05	0.0002079680264675636	8.907854334579257e-05	-6.261071933601408e-06	-8.608892840971337e-06
-3.66164143988934e-05	8.907854334579684e-05	4.6455362189046436e-05	-2.4166333614153135e-06	-4.9241228485738274e-06
3.2792061676648674e-06	-6.261071933599017e-06	-2.4166333614136584e-06	3.2971299830183826e-07	1.9446551229418968e-07
3.1326978374877192e-06	-8.608892840972704e-06	-4.9241228485747354e-06	1.9446551229437007e-07	7.105443131016032e-07


## Cosmic Parameters + Galaxy Bias FISHER MATRIX

In [435]:
np.shape(dsignal_dgbias50), np.shape(dsignal_dgbias30), np.shape(dsignal_dcosmic)

((4, 19, 648), (4, 19, 648), (5, 19, 612))

In [436]:
dsignal_dpars50x50 = np.concatenate([dsignal_dgbias50_, dsignal_dcosmic50])
dsignal_dpars30x70 = np.concatenate([dsignal_dgbias30_, dsignal_dcosmic30])

dsignal_dgbias50 = np.delete(dsignal_dgbias50, ind_to_remove, axis=2)
dsignal_dgbias30 = np.delete(dsignal_dgbias30, ind_to_remove, axis=2)

dsignal_dpars_joint = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dpars50x50), np.shape(dsignal_dpars30x70), np.shape(dsignal_dpars_joint)

((9, 19, 324), (9, 19, 324), (13, 19, 612))

In [437]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dpars50x50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dpars30x70, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dpars_joint, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((9, 9), (9, 9), (13, 13))

In [438]:
np.linalg.det(FisherMatrixSKA_50x50), np.linalg.det(FisherMatrixSKA_30x70), np.linalg.det(FisherMatrixSKA_joint)

(2.978620645887647e+53, 6.949421003610326e+53, 6.821016502284641e+83)

In [439]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

25447240.777854692	7515169.987835224	-31713835.347808655	-9110094.934605015	2177814.361231719	1105856.1562454673	-932413.0557254107	-6618981.011715738	-2933471.9958783127
7515169.987835222	2808444.203160712	-9110094.934605015	-3225463.7597114304	1072997.0184735924	504090.3908093623	-332884.4190554553	-3396522.347946506	-474952.963605881
-31713835.347809885	-9110094.93460506	48443965.55355428	14405111.655087853	2321035.4568680176	305381.6409740199	1695881.1591249863	-10068274.409692686	11822762.336750662
-9110094.934605056	-3225463.7597113927	14405111.655087844	5400372.421185124	629545.2719306645	69738.45404022718	496145.3945584891	-2793044.5741037377	3003394.232552508
2177814.3612309718	1072997.0184736014	2321035.456869026	629545.2719306569	5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1105856.1562451809	504090.390809364	305381.6409744265	69738.45404022578	1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.57

In [440]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

33016651.384679236	8332488.267908776	-50360159.32634173	-12901331.246122913	1874549.3290753085	1015691.4324662106	-994719.3725617193	-5479385.917692468	-3961393.346895395
8332488.267908773	2733114.2927259943	-12901331.246122926	-4149358.7031630417	846945.2037477822	422633.8689456244	-337296.306303709	-2592730.776060814	-779727.1200034039
-50360159.32633923	-12901331.24612243	84500438.42434922	23254467.951169528	2067729.6980300494	117760.24214361416	1973312.5021055618	-9524773.356651565	13038348.119257785
-12901331.246122435	-4149358.7031629556	23254467.951169528	8118604.127277631	736809.8546538168	81090.78851551346	578643.8650978748	-3279873.2486417764	3469697.352632582
1874549.3290768762	846945.2037479944	2067729.6980268739	736809.8546534531	5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1015691.4324669703	422633.86894572224	117760.24214217364	81090.7885153587	1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473

In [441]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

239836850.62495834	48697266.23710122	170533246.63374412	18168276.35035656	-86354322.11230232	-19768479.423442256	-358913738.8152637	-53284837.76877848	2194809.158537092	1097536.5806002778	-870414.4301893655	-6725098.400473181	-3369598.640316342
48697266.23710123	14351933.217645446	18168276.350356564	1046135.4021589902	-19768479.423442263	-6699195.871897356	-53284837.76877847	-10075359.004489357	752103.1840154352	363707.8894107544	-259862.91652634702	-2334727.5858235816	-659403.3985773692
170533247.26709872	18168276.39835402	393454505.2658195	64371510.07652239	-22195058.948971387	1896947.293044668	-587487014.3914438	-91855671.97164471	849139.5028543855	295817.07501475874	62472.274187524185	-3072626.3861645493	2542556.6771948826
18168276.39835402	1046135.4053310191	64371510.07652238	13857853.288172426	1896947.2930446756	1833783.048967673	-91855671.97164473	-17982927.874865077	180258.58592617506	46624.41439750799	63031.69997074727	-703771.1324359342	642805.4906908451
-86354321.53384532	-1

In [442]:
dsignal_dpars_joint_ = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic_multi])

np.shape(dsignal_dpars_joint_)

(18, 19, 612)

In [443]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dpars_joint_, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



In [444]:
np.shape(FisherMatrixSKA_joint_), np.linalg.det(FisherMatrixSKA_joint_)

((18, 18), 1.2322517742426492e+120)

In [445]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

239836850.62495834	48697266.23710122	170533246.63374412	18168276.35035656	-86354322.11230232	-19768479.423442256	-358913738.8152637	-53284837.76877848	354064026.31351715	176246927.39882022	-136201093.8359703	-1077569687.179304	208494937.07153094	-351869217.15498	-175149390.8182199	135330679.40578094	1070844588.7788311	-211864535.71184742
48697266.23710123	14351933.217645446	18168276.350356564	1046135.4021589902	-19768479.423442263	-6699195.871897356	-53284837.76877847	-10075359.004489357	59329304.975869335	29449576.562688127	-22535843.56127617	-180599721.44593447	28049502.7610185	-58577201.7918539	-29085868.673277378	22275980.644749835	178264993.86011094	-28708906.15959587
170533247.26709872	18168276.39835402	393454505.2658195	64371510.07652239	-22195058.948971387	1896947.293044668	-587487014.3914438	-91855671.97164471	426437443.39795786	214711291.63732675	-169688034.16453156	-1279274126.63607	285576678.0363355	-425588303.89510363	-214415474.56231198	169750506.43871903	1276201500.24990

#  Joint Covariance Matrix 50x30 [all] MONOPOLE ONLY

## Check Cauchy-Schwarz inequality 

The Cauchy-Schwarz inequality implies:

$$ \mathrm{COV}(X,Y) \leq \sqrt{\mathrm{VAR}(X)\cdot\mathrm{VAR}(Y)}$$

In [261]:
z_bins = np.array([0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])

In [262]:
CovMatrixSKA_joint_ = fm.cov_matrix_joint(d=dist[0], z_bins=z_bins, small_off_diagonal=True, rtol=1e-4, splits=[50,30], contributions='all')


 ... Loading Covariance Matrices (Joint-splittings Analysis) [all] [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 



In [265]:
n_sep = len(dist[0])
n_dim = len(CovMatrixSKA_50x50[0,0])

ind_to_remove = list(range(3*n_sep,9*n_sep))+list(range(3*n_sep+n_dim,9*n_sep+n_dim))

print(n_sep, n_dim)

36 324


In [266]:
print('\n ... EXTRACT THE MONOPOLES JOINT MATRIX from 50x50 split ... \n')

CovMatrixSKA_joint = np.delete(CovMatrixSKA_joint_, ind_to_remove, axis=1)
CovMatrixSKA_joint = np.delete(CovMatrixSKA_joint, ind_to_remove, axis=2)

np.shape(CovMatrixSKA_joint_), np.shape(CovMatrixSKA_joint)


 ... EXTRACT THE MONOPOLES JOINT MATRIX from 50x50 split ... 



((19, 648, 648), (19, 216, 216))

In [268]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol = 1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [269]:
np.shape(CovMatrixSKA_50x50), np.shape(CovMatrixSKA_30x70), np.shape(CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 216, 216))

## Checking directly from Joint Cov Matrix 50x30

Compare off diagonal elements (covariances) versus the correspondent diagonal ones (variances).

In [270]:
np.shape(CovMatrixSKA_joint), type(CovMatrixSKA_joint)

((19, 216, 216), numpy.ndarray)

In [271]:
2*36*9-36

612

In [273]:
print(' Monopole BB (d=d), 50x50 = ', CovMatrixSKA_joint[0][0,0],'\n','Monopole BB (d=d), 30x70 = ', CovMatrixSKA_joint[0][3*n_sep,3*n_sep] ) 

 Monopole BB (d=d), 50x50 =  1.1406613424594502e-05 
 Monopole BB (d=d), 30x70 =  2.728761334220845e-05


In [274]:
print(' Monopole BB (d=d), 50x30 = ', CovMatrixSKA_joint[0][0,3*n_sep],'\n', 'Monopole BB (d=d), 30x50 = ', CovMatrixSKA_joint[0][3*n_sep,0] ) 

 Monopole BB (d=d), 50x30 =  1.761081877983889e-05 
 Monopole BB (d=d), 30x50 =  1.761081877983889e-05


In [275]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][3*n_sep,3*n_sep]) >= CovMatrixSKA_joint[k][0,3*n_sep], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][3*n_sep,3*n_sep]) >= CovMatrixSKA_joint[k][3*n_sep,0])

0.15 True True
0.25 True True
0.35 True True
0.45 True True
0.55 True True
0.65 True True
0.75 True True
0.85 True True
0.95 True True
1.05 True True
1.15 True True
1.25 True True
1.35 True True
1.45 True True
1.55 True True
1.65 True True
1.75 True True
1.85 True True
1.95 True True


In [287]:
i_element = 5
j_element = i_element
l = 3*n_sep

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  4.218886417437754e-06 
 Element (d=d), 30x70 =  1.0092376870537708e-05
 Element (d=d), 50x30 =  6.513999438089945e-06 
 Element (d=d), 30x50 =  6.513999438089945e-06 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [282]:
print(CovMatrixSKA_50x50[0][24,24], CovMatrixSKA_50x50[0][36,36])
print(CovMatrixSKA_50x50[0][24,36], CovMatrixSKA_50x50[0][36,24])

5.093876193886468e-07 1.0577598333198072e-06
5.353246240816097e-08 5.353246240816097e-08


In [283]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_50x50[k][10,25]**2 <= CovMatrixSKA_50x50[k][10,10]*CovMatrixSKA_50x50[k][25,25])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


In [284]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_30x70[k][18,30]**2 <= CovMatrixSKA_30x70[k][18,18]*CovMatrixSKA_30x70[k][30,30])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


## Test Fisher Matrix

### Load the derivatives

In [362]:
with open('derivatives.pkl', 'rb') as File:
    derivatives_dict = pk.load(File)
    
derivatives_50x50 = derivatives_dict['split_50x50']
derivatives_30x70 = derivatives_dict['split_30x70']

In [363]:
dsignal_dsbias50 = derivatives_50x50['dsignal_dsbias']

dsignal_dgbias50_ = derivatives_50x50['dsignal_dgbias']

dsignal_dcosmic50 = derivatives_50x50['dsignal_dcosmic']

dsignal_dnum50 = derivatives_50x50['dsignal_dnumev']

np.shape(dsignal_dsbias50), np.shape(dsignal_dgbias50_), np.shape(dsignal_dcosmic50), np.shape(dsignal_dnum50)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [364]:
dsignal_dsbias30 = derivatives_30x70['dsignal_dsbias']

dsignal_dgbias30_ = derivatives_30x70['dsignal_dgbias']

dsignal_dcosmic30 = derivatives_30x70['dsignal_dcosmic']

dsignal_dnum30 = derivatives_30x70['dsignal_dnumev']

np.shape(dsignal_dsbias30), np.shape(dsignal_dgbias30_), np.shape(dsignal_dcosmic30), np.shape(dsignal_dnum30)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [365]:
dsignal_dtheta50 = np.concatenate([dsignal_dsbias50, dsignal_dnum50, dsignal_dgbias50_, dsignal_dcosmic50])

dsignal_dtheta50 = np.delete(dsignal_dtheta50, ind_to_remove[:int(len(ind_to_remove)/2)], axis=2)

np.shape(dsignal_dtheta50)

(21, 19, 108)

In [366]:
dsignal_dtheta30 = np.concatenate([dsignal_dsbias30, dsignal_dnum30, dsignal_dgbias30_, dsignal_dcosmic30])

dsignal_dtheta30 = np.delete(dsignal_dtheta30, ind_to_remove[:int(len(ind_to_remove)/2)], axis=2)

np.shape(dsignal_dtheta30)

(21, 19, 108)

In [367]:
dsignal_dsB50 = np.concatenate([dsignal_dsbias50[:4], np.zeros_like(dsignal_dsbias50[:4])], axis=2)
dsignal_dsB30 = np.concatenate([np.zeros_like(dsignal_dsbias30[:4]), dsignal_dsbias30[:4]], axis=2)

dsignal_dsM = np.concatenate([dsignal_dsbias50[4:], dsignal_dsbias30[4:]], axis=2)
dsignal_dnum = np.concatenate([dsignal_dnum50, dsignal_dnum30], axis=2)
dsignal_dgbias50 = np.concatenate([dsignal_dgbias50_, np.zeros_like(dsignal_dgbias50_[:4])], axis=2)
dsignal_dgbias30 = np.concatenate([np.zeros_like(dsignal_dgbias30_[:4]), dsignal_dgbias30_], axis=2)
dsignal_dcosmic = np.concatenate([dsignal_dcosmic50, dsignal_dcosmic30], axis=2)

In [368]:
dsignal_dtheta_multi = np.concatenate([dsignal_dsB50, dsignal_dsB30, dsignal_dsM, dsignal_dnum, dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

dsignal_dtheta_multi = np.delete(dsignal_dtheta_multi, ind_to_remove, axis=2)
dsignal_dcosmic = np.delete(dsignal_dcosmic, ind_to_remove, axis=2)

np.shape(dsignal_dtheta_multi), np.shape(dsignal_dcosmic)

((29, 19, 216), (5, 19, 216))

## Cosmic Parameters Fisher Matrices

In [369]:
Inv_CovMatrixSKA_joint = np.array([np.linalg.pinv(CovMatrixSKA_joint[i]) for i,k in enumerate(z_bins)])

np.shape(Inv_CovMatrixSKA_50x50), np.shape(Inv_CovMatrixSKA_30x70), np.shape(Inv_CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 216, 216))

In [370]:
np.linalg.det(CovMatrixSKA_joint), np.linalg.det(Inv_CovMatrixSKA_joint)

/Users/danielsb/opt/anaconda3/envs/camb/lib/python3.9/site-packages/numpy/linalg/linalg.py:2139: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


(array([-0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
         0.,  0.,  0.,  0.,  0.,  0.]),
 array([ inf, -inf,  inf,  inf,  inf, -inf, -inf, -inf,  inf, -inf, -inf,
        -inf,  inf, -inf,  inf, -inf,  inf,  inf, -inf]))

In [371]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(Inv_CovMatrixSKA_50x50[n], Inv_CovMatrixSKA_50x50[n].T, rtol=1e-4), 
          np.allclose(Inv_CovMatrixSKA_30x70[n], Inv_CovMatrixSKA_30x70[n].T, rtol=1e-4),
          np.allclose(Inv_CovMatrixSKA_joint[n], Inv_CovMatrixSKA_joint[n].T, rtol=1e-1))

0 True True True
1 True True True
2 True True True
3 True True True
4 True True True
5 True True True
6 True True True
7 True True True
8 True True True
9 True True True
10 True True True
11 True True True
12 True True True
13 True True True
14 True True True
15 True True True
16 True True True
17 True True True
18 True True True


In [372]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dcosmic50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dcosmic30, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dcosmic, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((5, 5), (5, 5), (5, 5))

In [373]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.579750899
496226.5426212941	96541.34153693542	269437.40309821675	-1928441.3798997358	1401221.6430270765
-19413664.00695389	-6055929.2249728255	-1928441.3798995449	80306511.07905662	-23831659.69196253
6641394.102308079	2108424.579750879	1401221.643026993	-23831659.691962913	13961632.845623164


In [374]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473726.474570977
647009.0811369446	138897.17270890815	310960.1564787712	-2499890.3801519116	1671688.8824865094
-22479456.449508905	-6988605.066998642	-2499890.3801519973	92043357.18543775	-28355007.711743876
7855159.798259621	2473726.4745712737	1671688.8824863958	-28355007.71174604	15915477.89555201


In [375]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

2133687.517889752	794008.4126335182	-214003.91158581895	-8035716.738384281	929019.9318244644
794008.4126334895	327545.4341689119	-112992.61608574488	-2689073.3702787836	436037.0520637652
-214003.9115858422	-112992.61608577095	104503.77131064444	749383.9867290747	45704.395473502926
-8035716.738384091	-2689073.3702788223	749383.9867289994	37848641.06224067	-2271371.105499264
929019.9318244952	436037.05206380074	45704.39547347172	-2271371.1054993747	2268551.9689084417


In [376]:
np.shape(dsignal_dcosmic50)

(5, 19, 324)

In [377]:
dsignal_dcosmic50_ = np.concatenate([dsignal_dcosmic50, np.zeros_like(dsignal_dcosmic30)], axis=2)
dsignal_dcosmic30_ = np.concatenate([np.zeros_like(dsignal_dcosmic50), dsignal_dcosmic30], axis=2)

dsignal_dcosmic_multi = np.concatenate([dsignal_dcosmic50_, dsignal_dcosmic30_])

dsignal_dcosmic_multi = np.delete(dsignal_dcosmic_multi, ind_to_remove, axis=2)

np.shape(dsignal_dcosmic50_), np.shape(dsignal_dcosmic30_), np.shape(dsignal_dcosmic_multi)

((5, 19, 648), (5, 19, 648), (10, 19, 216))

In [378]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dcosmic_multi, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



In [379]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

134729160.300716	57109921.53384178	-29788469.31991109	-473442075.61174065	79870391.37836367	-133467174.32455377	-56610195.12004801	29587204.580697846	468831082.078828	-78891422.29997244
57109921.53089588	26769409.95647544	-17130306.812577907	-179896143.68667725	33314639.900238015	-56610420.57188802	-26547509.78192928	17014988.954025403	178275980.74235058	-32898662.203369576
-29788469.322782144	-17130306.81487867	17972160.927427996	85008041.77905238	-361065.69714028435	29587851.342460506	17015222.480170462	-17850773.83106626	-84402913.73789048	353039.3469264778
-473442075.7358447	-179896143.76011208	85008041.82167992	2050390518.330116	-252776468.74890205	468830028.36223376	178274754.81959513	-84400490.42111348	-2029150152.5229979	249806544.0474036
79870391.35219382	33314639.890692674	-361065.68700744415	-252776468.56808776	151234843.13208744	-79304694.71774523	-33067649.559050955	407988.2504935548	251217062.32164207	-149418649.0195276
-133467174.32453793	-56610420.57482915	29587851.3395

In [380]:
np.linalg.det(FisherMatrixSKA_joint), np.linalg.det(FisherMatrixSKA_joint_)

(3.0514196733892736e+27, 4.807332339808402e+63)

In [381]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in np.linalg.inv(FisherMatrixSKA_joint)]))

9.452654473332236e-05	-0.00023606577037501786	-0.00011509986790939524	6.505979633441481e-06	1.5496423701014465e-05
-0.00023606577037502225	0.0006135473952054965	0.0003074300634218457	-1.5174227242871954e-05	-4.264253120032149e-05
-0.00011509986790944789	0.00030743006342197885	0.00016844097889510328	-7.288869414652904e-06	-2.264662583433861e-05
6.505979633441398e-06	-1.5174227242871522e-05	-7.288869414649528e-06	5.297161770053184e-07	9.295162626043739e-07
1.549642370101744e-05	-4.2642531200328464e-05	-2.2646625834332474e-05	9.295162626045656e-07	3.6779249254515622e-06


## Cosmic Parameters + Galaxy Bias FISHER MATRIX

In [382]:
dsignal_dpars50x50 = np.concatenate([dsignal_dgbias50_, dsignal_dcosmic50])
dsignal_dpars30x70 = np.concatenate([dsignal_dgbias30_, dsignal_dcosmic30])

dsignal_dgbias50 = np.delete(dsignal_dgbias50, ind_to_remove, axis=2)
dsignal_dgbias30 = np.delete(dsignal_dgbias30, ind_to_remove, axis=2)

dsignal_dpars_joint = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dpars50x50), np.shape(dsignal_dpars30x70), np.shape(dsignal_dpars_joint)

((9, 19, 324), (9, 19, 324), (13, 19, 216))

In [383]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dpars50x50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dpars30x70, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dpars_joint, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((9, 9), (9, 9), (13, 13))

In [384]:
np.linalg.det(FisherMatrixSKA_50x50), np.linalg.det(FisherMatrixSKA_30x70), np.linalg.det(FisherMatrixSKA_joint)

(2.978620645887647e+53, 6.949421003610326e+53, 4.109011071237484e+78)

In [385]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

25447240.777854692	7515169.987835224	-31713835.347808655	-9110094.934605015	2177814.361231719	1105856.1562454673	-932413.0557254107	-6618981.011715738	-2933471.9958783127
7515169.987835222	2808444.203160712	-9110094.934605015	-3225463.7597114304	1072997.0184735924	504090.3908093623	-332884.4190554553	-3396522.347946506	-474952.963605881
-31713835.347809885	-9110094.93460506	48443965.55355428	14405111.655087853	2321035.4568680176	305381.6409740199	1695881.1591249863	-10068274.409692686	11822762.336750662
-9110094.934605056	-3225463.7597113927	14405111.655087844	5400372.421185124	629545.2719306645	69738.45404022718	496145.3945584891	-2793044.5741037377	3003394.232552508
2177814.3612309718	1072997.0184736014	2321035.456869026	629545.2719306569	5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1105856.1562451809	504090.390809364	305381.6409744265	69738.45404022578	1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.57

In [386]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

33016651.384679236	8332488.267908776	-50360159.32634173	-12901331.246122913	1874549.3290753085	1015691.4324662106	-994719.3725617193	-5479385.917692468	-3961393.346895395
8332488.267908773	2733114.2927259943	-12901331.246122926	-4149358.7031630417	846945.2037477822	422633.8689456244	-337296.306303709	-2592730.776060814	-779727.1200034039
-50360159.32633923	-12901331.24612243	84500438.42434922	23254467.951169528	2067729.6980300494	117760.24214361416	1973312.5021055618	-9524773.356651565	13038348.119257785
-12901331.246122435	-4149358.7031629556	23254467.951169528	8118604.127277631	736809.8546538168	81090.78851551346	578643.8650978748	-3279873.2486417764	3469697.352632582
1874549.3290768762	846945.2037479944	2067729.6980268739	736809.8546534531	5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1015691.4324669703	422633.86894572224	117760.24214217364	81090.7885153587	1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473

In [387]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

92066506.36463049	26920303.855945222	-15342405.252876662	-6656001.943252067	-56902183.571835	-16626971.718238655	-21945128.911250416	-4068466.4882647437	1332585.6925930122	593108.938667171	-311007.5775627017	-4322794.222278359	630513.5381725173
26920303.855945222	10198339.605717944	-6656001.943252066	-2914131.820052219	-16626971.718238655	-6297889.97474721	-4068466.4882647446	-1097314.1163115448	569106.6792123744	254879.21872294825	-137185.29296954433	-1838826.4266961366	182875.69014475791
-15342405.2546295	-6656001.94332883	60981355.31990123	12966310.573607504	20652150.389404207	6199905.061081541	-78198565.31662098	-14667962.713502007	-369657.2337270996	-160982.47376826243	76836.13927080088	1224708.1824009353	519903.6029943622
-6656001.943328827	-2914131.820039441	12966310.57360752	3694245.1182399895	6199905.061081552	2214836.9799904865	-14667962.713501988	-3335463.361361189	-134718.12225294244	-62195.51599628249	37893.38293994561	431802.06247003184	160859.91971105168
-56902183.572485

In [388]:
dsignal_dpars_joint_ = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic_multi])

np.shape(dsignal_dpars_joint_)

(18, 19, 216)

In [389]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dpars_joint_, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



In [390]:
np.shape(FisherMatrixSKA_joint_), np.linalg.det(FisherMatrixSKA_joint_)

((18, 18), 3.3216627958448616e+111)

In [391]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

92066506.36463049	26920303.855945222	-15342405.252876662	-6656001.943252067	-56902183.571835	-16626971.718238655	-21945128.911250416	-4068466.4882647437	91670575.53544714	42717960.99014189	-26796840.411836803	-290006127.79184276	47572413.241915524	-90337989.8428541	-42124852.0514747	26485832.8342741	285683333.5695643	-46941899.703743
26920303.855945222	10198339.605717944	-6656001.943252066	-2914131.820052219	-16626971.718238655	-6297889.97474721	-4068466.4882647446	-1097314.1163115448	23368034.837180924	10991956.332475802	-7113388.166193549	-73429113.68377201	9162550.227619926	-22798928.157968547	-10737077.113752859	6976202.873224005	71590287.25707585	-8979674.537475172
-15342405.2546295	-6656001.94332883	60981355.31990123	12966310.573607504	20652150.389404207	6199905.061081541	-78198565.31662098	-14667962.713502007	25209953.229723312	12292587.533625571	-8883655.048157772	-77015777.10726714	21986271.807046533	-25579610.463450417	-12453570.007393837	8960491.18742857	78240485.28966808	-2

#  Joint Covariance Matrix 50x70 [all] REMOVE ONE HEXADECAPOLE

## Check Cauchy-Schwarz inequality 

The Cauchy-Schwarz inequality implies:

$$ \mathrm{COV}(X,Y) \leq \sqrt{\mathrm{VAR}(X)\cdot\mathrm{VAR}(Y)}$$

In [14]:
z_bins = np.array([0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05, 1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95])

n_sep = len(dist[0])

ind_to_remove = list(range(7*n_sep,8*n_sep))


In [15]:
CovMatrixSKA_joint_ = fm.cov_matrix_joint(d=dist[0], z_bins=z_bins, small_off_diagonal=True, rtol=1e-4, splits=[50,70], contributions='all')


 ... Loading Covariance Matrices (Joint-splittings Analysis) [all] [MONOBB, MONOBF, MONOFF, DIPBF, QUADBB, QUADBF, QUADFF, HEXAT, OCTBF] ... 



In [16]:
print('\n ... Remove the HEXADECAPOLE from 50x50 split ... \n')

CovMatrixSKA_joint = np.delete(CovMatrixSKA_joint_, ind_to_remove, axis=1)
CovMatrixSKA_joint = np.delete(CovMatrixSKA_joint, ind_to_remove, axis=2)

np.shape(CovMatrixSKA_joint_), np.shape(CovMatrixSKA_joint)


 ... Remove the HEXADECAPOLE from 50x50 split ... 



((19, 648, 648), (19, 612, 612))

In [17]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(CovMatrixSKA_joint[n], CovMatrixSKA_joint[n].T, rtol = 1e-20))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True


In [18]:
np.shape(CovMatrixSKA_50x50), np.shape(CovMatrixSKA_70x30), np.shape(CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 612, 612))

## Checking directly from Joint Cov Matrix 50x70

Compare off diagonal elements (covariances) versus the correspondent diagonal ones (variances).

In [19]:
np.shape(CovMatrixSKA_joint), type(CovMatrixSKA_joint)

((19, 612, 612), numpy.ndarray)

In [20]:
2*36*9-36

612

In [21]:
print(' Monopole BB (d=d), 50x50 = ', CovMatrixSKA_joint[0][0,0],'\n','Monopole BB (d=d), 70x30 = ', CovMatrixSKA_joint[0][324,324] ) 

 Monopole BB (d=d), 50x50 =  1.1406613424594502e-05 
 Monopole BB (d=d), 70x30 =  5.638417702019513e-07


In [22]:
print(' Monopole BB (d=d), 50x70 = ', CovMatrixSKA_joint[0][0,324],'\n', 'Monopole BB (d=d), 70x50 = ', CovMatrixSKA_joint[0][324,0] ) 

 Monopole BB (d=d), 50x70 =  5.613138257797223e-06 
 Monopole BB (d=d), 70x50 =  5.613138257797223e-06


In [23]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][0,324], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]) >= CovMatrixSKA_joint[k][324,0])

0.15 False False
0.25 False False
0.35 False False
0.45 False False
0.55 False False
0.65 False False
0.75 False False
0.85 False False
0.95 False False
1.05 False False
1.15 False False
1.25 False False
1.35 True True
1.45 True True
1.55 True True
1.65 True True
1.75 True True
1.85 True True
1.95 True True


In [24]:
for k,_ in enumerate(z_bins):
    print(z_bins[k], np.sqrt(CovMatrixSKA_joint[k][0,0] * CovMatrixSKA_joint[k][324,324]), CovMatrixSKA_joint[k][0,324])

0.15 2.536045170207484e-06 5.613138257797223e-06
0.25 1.1759044891878678e-06 2.4013378376522883e-06
0.35 7.450113415260696e-07 1.4245096642642512e-06
0.45 5.5031050034784e-07 9.948754287797074e-07
0.55 4.460848183034741e-07 7.6719573894467e-07
0.65 3.856208309751128e-07 6.332227301086177e-07
0.75 3.504145653179042e-07 5.499569580095534e-07
0.85 3.321896418365526e-07 4.975540816320136e-07
0.95 3.2770012949669244e-07 4.66089114352573e-07
1.05 3.370420312793053e-07 4.505703449219131e-07
1.15 3.6319962723115915e-07 4.4874624504609085e-07
1.25 4.150521430880537e-07 4.606856977823527e-07
1.35 5.129675776023526e-07 4.887762274491425e-07
1.45 7.00790038571815e-07 5.380493619721122e-07
1.55 1.0957302823682655e-06 6.200987233643109e-07
1.65 1.9845528176588043e-06 7.545578343527732e-07
1.75 4.153655618440485e-06 9.79432459179411e-07
1.85 9.983327692749943e-06 1.3733660181748274e-06
1.95 2.6858881386093924e-05 2.090715609007057e-06


In [25]:
print(' MonoBQuadBF (d=d), 50x50 = ', CovMatrixSKA_joint[0][179,179],'\n','MonoBQuadBF (d=d) (d=d), 70x30 = ', CovMatrixSKA_joint[0][179+324,179+324] )
print(' Monopole BB (d=d), 50x70 = ', CovMatrixSKA_joint[0][179,179+324],'\n', 'Monopole BB (d=d), 70x50 = ', CovMatrixSKA_joint[0][179+324,179] )  

 MonoBQuadBF (d=d), 50x50 =  1.9425535215326826e-06 
 MonoBQuadBF (d=d) (d=d), 70x30 =  1.5008788056283315e-07
 Monopole BB (d=d), 50x70 =  1.0518285311743783e-06 
 Monopole BB (d=d), 70x50 =  1.0518285311743783e-06


In [26]:
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][179,179+324], 
          np.sqrt(CovMatrixSKA_joint[k][179,179] * CovMatrixSKA_joint[k][179+324,179+324]) >= CovMatrixSKA_joint[k][324+179,179])

False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
True True
True True
True True
True True
True True
True True


In [27]:
len(CovMatrixSKA_30x50[0])

NameError: name 'CovMatrixSKA_70x50' is not defined

In [28]:
i_element = 288
j_element = i_element
l = int((2*36*9-36)/2)

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 70x30 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x70 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 70x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  5.056783304171943e-06 
 Element (d=d), 70x30 =  3.42118942960755e-09
 Element (d=d), 50x70 =  0.0 
 Element (d=d), 70x50 =  0.0 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [ ]:
i_element = 56
j_element = i_element
l = len(CovMatrixSKA_50x50[0])

print(' Element (d=d), 50x50 = ', CovMatrixSKA_joint[0][i_element,j_element],'\n','Element (d=d), 30x70 = ', CovMatrixSKA_joint[0][i_element+l,j_element+l] )
print(' Element (d=d), 50x30 = ', CovMatrixSKA_joint[0][i_element,j_element+l],'\n', 'Element (d=d), 30x50 = ', CovMatrixSKA_joint[0][i_element+l,j_element],'\n' )  

print('Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) \n')
for k,_ in enumerate(z_bins):
    print(np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element,j_element+l], 
          np.sqrt(CovMatrixSKA_joint[k][i_element,j_element] * CovMatrixSKA_joint[k][i_element+l,j_element+l]) >= CovMatrixSKA_joint[k][i_element+l,j_element])

 Element (d=d), 50x50 =  6.551780712795955e-08 
 Element (d=d), 30x70 =  2.037598373627894e-08
 Element (d=d), 50x30 =  3.645579650706823e-08 
 Element (d=d), 30x50 =  3.645579650706823e-08 

Check COV(X,Y)^2 <= SQRT(VAR(X) * VAR(Y)) 

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


In [29]:
print(CovMatrixSKA_50x50[0][24,24], CovMatrixSKA_50x50[0][36,36])
print(CovMatrixSKA_50x50[0][24,36], CovMatrixSKA_50x50[0][36,24])

5.093876193886468e-07 1.0577598333198072e-06
5.353246240816097e-08 5.353246240816097e-08


In [30]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_50x50[k][10,25]**2 <= CovMatrixSKA_50x50[k][10,10]*CovMatrixSKA_50x50[k][25,25])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


In [31]:
for k,_ in enumerate(z_bins):
    print('z = ',_, '| check: ', CovMatrixSKA_70x30[k][18,30]**2 <= CovMatrixSKA_70x30[k][18,18]*CovMatrixSKA_70x30[k][30,30])

z =  0.15 | check:  True
z =  0.25 | check:  True
z =  0.35 | check:  True
z =  0.45 | check:  True
z =  0.55 | check:  True
z =  0.65 | check:  True
z =  0.75 | check:  True
z =  0.85 | check:  True
z =  0.95 | check:  True
z =  1.05 | check:  True
z =  1.15 | check:  True
z =  1.25 | check:  True
z =  1.35 | check:  True
z =  1.45 | check:  True
z =  1.55 | check:  True
z =  1.65 | check:  True
z =  1.75 | check:  True
z =  1.85 | check:  True
z =  1.95 | check:  True


## Test Fisher Matrix

### Load the derivatives

In [33]:
with open('derivatives.pkl', 'rb') as File:
    derivatives_dict = pk.load(File)
    
derivatives_50x50 = derivatives_dict['split_50x50']
derivatives_30x70 = derivatives_dict['split_30x70']
derivatives_70x30 = derivatives_dict['split_70x30']


In [34]:
dsignal_dsbias50 = derivatives_50x50['dsignal_dsbias']

dsignal_dgbias50_ = derivatives_50x50['dsignal_dgbias']

dsignal_dcosmic50 = derivatives_50x50['dsignal_dcosmic']

dsignal_dnum50 = derivatives_50x50['dsignal_dnumev']

np.shape(dsignal_dsbias50), np.shape(dsignal_dgbias50_), np.shape(dsignal_dcosmic50), np.shape(dsignal_dnum50)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [35]:
dsignal_dsbias30 = derivatives_30x70['dsignal_dsbias']

dsignal_dgbias30_ = derivatives_30x70['dsignal_dgbias']

dsignal_dcosmic30 = derivatives_30x70['dsignal_dcosmic']

dsignal_dnum30 = derivatives_30x70['dsignal_dnumev']

np.shape(dsignal_dsbias30), np.shape(dsignal_dgbias30_), np.shape(dsignal_dcosmic30), np.shape(dsignal_dnum30)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [36]:
dsignal_dsbias70 = derivatives_70x30['dsignal_dsbias']

dsignal_dgbias70_ = derivatives_70x30['dsignal_dgbias']

dsignal_dcosmic70 = derivatives_70x30['dsignal_dcosmic']

dsignal_dnum70 = derivatives_70x30['dsignal_dnumev']

np.shape(dsignal_dsbias70), np.shape(dsignal_dgbias70_), np.shape(dsignal_dcosmic70), np.shape(dsignal_dnum70)

((8, 19, 324), (4, 19, 324), (5, 19, 324), (4, 19, 324))

In [37]:
dsignal_dtheta50 = np.concatenate([dsignal_dsbias50, dsignal_dnum50, dsignal_dgbias50_, dsignal_dcosmic50])

np.shape(dsignal_dtheta50)

(21, 19, 324)

In [38]:
dsignal_dtheta30 = np.concatenate([dsignal_dsbias30, dsignal_dnum30, dsignal_dgbias30_, dsignal_dcosmic30])

np.shape(dsignal_dtheta30)

(21, 19, 324)

In [39]:
dsignal_dtheta70 = np.concatenate([dsignal_dsbias70, dsignal_dnum70, dsignal_dgbias70_, dsignal_dcosmic70])

np.shape(dsignal_dtheta70)

(21, 19, 324)

In [40]:
dsignal_dsB50 = np.concatenate([dsignal_dsbias50[:4], np.zeros_like(dsignal_dsbias50[:4])], axis=2)
dsignal_dsB30 = np.concatenate([np.zeros_like(dsignal_dsbias30[:4]), dsignal_dsbias30[:4]], axis=2)
dsignal_dsB70 = np.concatenate([np.zeros_like(dsignal_dsbias70[:4]), dsignal_dsbias70[:4]], axis=2)


dsignal_dsM = np.concatenate([dsignal_dsbias50[4:], dsignal_dsbias70[4:]], axis=2)
dsignal_dnum = np.concatenate([dsignal_dnum50, dsignal_dnum70], axis=2)
dsignal_dgbias50 = np.concatenate([dsignal_dgbias50_, np.zeros_like(dsignal_dgbias50_[:4])], axis=2)
dsignal_dgbias30 = np.concatenate([np.zeros_like(dsignal_dgbias70_[:4]), dsignal_dgbias70_], axis=2)
dsignal_dgbias70 = np.concatenate([np.zeros_like(dsignal_dgbias70_[:4]), dsignal_dgbias70_], axis=2)
dsignal_dcosmic = np.concatenate([dsignal_dcosmic50, dsignal_dcosmic70], axis=2)

In [41]:
dsignal_dtheta_multi = np.concatenate([dsignal_dsB50, dsignal_dsB70, dsignal_dsM, dsignal_dnum, dsignal_dgbias50, dsignal_dgbias70, dsignal_dcosmic])

dsignal_dtheta_multi = np.delete(dsignal_dtheta_multi, ind_to_remove, axis=2)
dsignal_dcosmic = np.delete(dsignal_dcosmic, ind_to_remove, axis=2)

np.shape(dsignal_dtheta_multi), np.shape(dsignal_dcosmic)

((29, 19, 612), (5, 19, 612))

## Cosmic Parameters Fisher Matrices

In [43]:
Inv_CovMatrixSKA_joint = np.array([np.linalg.pinv(CovMatrixSKA_joint[i]) for i,k in enumerate(z_bins)])

np.shape(Inv_CovMatrixSKA_50x50), np.shape(Inv_CovMatrixSKA_70x30), np.shape(Inv_CovMatrixSKA_joint)

((19, 324, 324), (19, 324, 324), (19, 612, 612))

In [45]:
for n, z in enumerate(z_bins):
    print(n, np.allclose(Inv_CovMatrixSKA_50x50[n], Inv_CovMatrixSKA_50x50[n].T, rtol=1e-4), 
          np.allclose(Inv_CovMatrixSKA_70x30[n], Inv_CovMatrixSKA_70x30[n].T, rtol=1e-4),
          np.allclose(Inv_CovMatrixSKA_joint[n], Inv_CovMatrixSKA_joint[n].T, rtol=1e-1))

0 True True False
1 True True False
2 True True False
3 True True False
4 True True False
5 True True False
6 True True False
7 True True False
8 True True False
9 True True False
10 True True False
11 True True False
12 True True False
13 True True False
14 True True False
15 True True False
16 True True False
17 True True False
18 True True False


In [47]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dcosmic50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_70x30 = fm.fisher_matrix(d_signal=dsignal_dcosmic30, inv_cov=Inv_CovMatrixSKA_70x30, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dcosmic, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_70x30), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((5, 5), (5, 5), (5, 5))

In [48]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.579750899
496226.5426212941	96541.34153693542	269437.40309821675	-1928441.3798997358	1401221.6430270765
-19413664.00695389	-6055929.2249728255	-1928441.3798995449	80306511.07905662	-23831659.69196253
6641394.102308079	2108424.579750879	1401221.643026993	-23831659.691962913	13961632.845623164


In [49]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

NameError: name 'FisherMatrixSKA_30x70' is not defined

In [ ]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

6251295.368837502	2026141.1179901503	687527.5698482153	-23482439.4055779	8178810.243777531
2026141.117990104	699264.2519198089	148489.37206981913	-7297374.249152005	2565456.666756784
687527.5698483501	148489.37206987408	327476.46734001266	-2658934.1807957934	1765010.6975681668
-23482439.405577064	-7297374.249151797	-2658934.180795723	95961186.90261878	-29573082.66786928
8178810.243777963	2565456.666756904	1765010.6975683053	-29573082.667871144	16756273.07661894


In [ ]:
np.shape(dsignal_dcosmic50)

(5, 19, 324)

In [ ]:
dsignal_dcosmic50_ = np.concatenate([dsignal_dcosmic50, np.zeros_like(dsignal_dcosmic30)], axis=2)
dsignal_dcosmic30_ = np.concatenate([np.zeros_like(dsignal_dcosmic50), dsignal_dcosmic30], axis=2)

dsignal_dcosmic_multi = np.concatenate([dsignal_dcosmic50_, dsignal_dcosmic30_])

dsignal_dcosmic_multi = np.delete(dsignal_dcosmic_multi, ind_to_remove, axis=2)

np.shape(dsignal_dcosmic50_), np.shape(dsignal_dcosmic30_), np.shape(dsignal_dcosmic_multi)

((5, 19, 648), (5, 19, 648), (10, 19, 612))

In [ ]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dcosmic_multi, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



In [ ]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

703872218.150775	319797474.72804046	-212245379.1844078	-2379823127.2053742	458883058.24678373	-700743671.1921499	-318511365.99167585	211724126.9584626	2368813880.4830008	-459591094.76312566
319797474.01400304	159990530.22853753	-124860282.52903669	-966871471.0093046	200114130.32489592	-318467386.31754535	-159384846.35101536	124514636.82020631	962572701.8048247	-200779356.09356847
-212245376.77242467	-124860281.60211782	136516642.24279994	589854186.6564937	-45670267.071075976	211615418.26900944	124472377.89075033	-136052453.63362306	-588064860.4949622	47237823.61807507
-2379823127.2024417	-966871472.6118709	589854192.4733844	10011341184.872478	-1454169894.2982576	2369131650.6487646	962834030.5090423	-588401478.1503981	-9967115738.710846	1456780753.308046
458883063.35614103	200114133.26483417	-45670270.49451372	-1454169908.1728764	789790775.2420641	-456879208.47708875	-199394060.95055488	45932501.20271988	1447408663.2480824	-786255353.2113926
-700743671.1870086	-318467387.0287613	2116154

In [ ]:
np.linalg.det(FisherMatrixSKA_joint), np.linalg.det(FisherMatrixSKA_joint_)

(1.6519686050731574e+29, 7.699403863968864e+68)

In [ ]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in np.linalg.inv(FisherMatrixSKA_joint)]))

4.1401384047626375e-05	-8.945846139545268e-05	-3.661641439888493e-05	3.2792061676656403e-06	3.1326978374865317e-06
-8.945846139546193e-05	0.0002079680264675636	8.907854334579257e-05	-6.261071933601408e-06	-8.608892840971337e-06
-3.66164143988934e-05	8.907854334579684e-05	4.6455362189046436e-05	-2.4166333614153135e-06	-4.9241228485738274e-06
3.2792061676648674e-06	-6.261071933599017e-06	-2.4166333614136584e-06	3.2971299830183826e-07	1.9446551229418968e-07
3.1326978374877192e-06	-8.608892840972704e-06	-4.9241228485747354e-06	1.9446551229437007e-07	7.105443131016032e-07


## Cosmic Parameters + Galaxy Bias FISHER MATRIX

In [ ]:
np.shape(dsignal_dgbias50), np.shape(dsignal_dgbias30), np.shape(dsignal_dcosmic)

((4, 19, 648), (4, 19, 648), (5, 19, 612))

In [ ]:
dsignal_dpars50x50 = np.concatenate([dsignal_dgbias50_, dsignal_dcosmic50])
dsignal_dpars30x70 = np.concatenate([dsignal_dgbias30_, dsignal_dcosmic30])

dsignal_dgbias50 = np.delete(dsignal_dgbias50, ind_to_remove, axis=2)
dsignal_dgbias30 = np.delete(dsignal_dgbias30, ind_to_remove, axis=2)

dsignal_dpars_joint = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic])

np.shape(dsignal_dpars50x50), np.shape(dsignal_dpars30x70), np.shape(dsignal_dpars_joint)

((9, 19, 324), (9, 19, 324), (13, 19, 612))

In [ ]:
FisherMatrixSKA_50x50 = fm.fisher_matrix(d_signal=dsignal_dpars50x50, inv_cov=Inv_CovMatrixSKA_50x50, z_bins=z_bins, bins=False)
FisherMatrixSKA_30x70 = fm.fisher_matrix(d_signal=dsignal_dpars30x70, inv_cov=Inv_CovMatrixSKA_30x70, z_bins=z_bins, bins=False)
FisherMatrixSKA_joint = fm.fisher_matrix(d_signal=dsignal_dpars_joint, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)

np.shape(FisherMatrixSKA_50x50), np.shape(FisherMatrixSKA_30x70), np.shape(FisherMatrixSKA_joint)



 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



((9, 9), (9, 9), (13, 13))

In [ ]:
np.linalg.det(FisherMatrixSKA_50x50), np.linalg.det(FisherMatrixSKA_30x70), np.linalg.det(FisherMatrixSKA_joint)

(2.978620645887647e+53, 6.949421003610326e+53, 6.821016502284641e+83)

In [ ]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_50x50]))

25447240.777854692	7515169.987835224	-31713835.347808655	-9110094.934605015	2177814.361231719	1105856.1562454673	-932413.0557254107	-6618981.011715738	-2933471.9958783127
7515169.987835222	2808444.203160712	-9110094.934605015	-3225463.7597114304	1072997.0184735924	504090.3908093623	-332884.4190554553	-3396522.347946506	-474952.963605881
-31713835.347809885	-9110094.93460506	48443965.55355428	14405111.655087853	2321035.4568680176	305381.6409740199	1695881.1591249863	-10068274.409692686	11822762.336750662
-9110094.934605056	-3225463.7597113927	14405111.655087844	5400372.421185124	629545.2719306645	69738.45404022718	496145.3945584891	-2793044.5741037377	3003394.232552508
2177814.3612309718	1072997.0184736014	2321035.456869026	629545.2719306569	5164002.026713927	1688886.113488194	496226.54262118076	-19413664.00695398	6641394.1023081085
1105856.1562451809	504090.390809364	305381.6409744265	69738.45404022578	1688886.1134881629	591306.3728766952	96541.34153689024	-6055929.224972827	2108424.57

In [ ]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_30x70]))

33016651.384679236	8332488.267908776	-50360159.32634173	-12901331.246122913	1874549.3290753085	1015691.4324662106	-994719.3725617193	-5479385.917692468	-3961393.346895395
8332488.267908773	2733114.2927259943	-12901331.246122926	-4149358.7031630417	846945.2037477822	422633.8689456244	-337296.306303709	-2592730.776060814	-779727.1200034039
-50360159.32633923	-12901331.24612243	84500438.42434922	23254467.951169528	2067729.6980300494	117760.24214361416	1973312.5021055618	-9524773.356651565	13038348.119257785
-12901331.246122435	-4149358.7031629556	23254467.951169528	8118604.127277631	736809.8546538168	81090.78851551346	578643.8650978748	-3279873.2486417764	3469697.352632582
1874549.3290768762	846945.2037479944	2067729.6980268739	736809.8546534531	5982702.933384105	1941106.7989340974	647009.0811368452	-22479456.449509744	7855159.7982589435
1015691.4324669703	422633.86894572224	117760.24214217364	81090.7885153587	1941106.7989341135	670904.4828461709	138897.17270884645	-6988605.066999086	2473

In [ ]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint]))

239836850.62495834	48697266.23710122	170533246.63374412	18168276.35035656	-86354322.11230232	-19768479.423442256	-358913738.8152637	-53284837.76877848	2194809.158537092	1097536.5806002778	-870414.4301893655	-6725098.400473181	-3369598.640316342
48697266.23710123	14351933.217645446	18168276.350356564	1046135.4021589902	-19768479.423442263	-6699195.871897356	-53284837.76877847	-10075359.004489357	752103.1840154352	363707.8894107544	-259862.91652634702	-2334727.5858235816	-659403.3985773692
170533247.26709872	18168276.39835402	393454505.2658195	64371510.07652239	-22195058.948971387	1896947.293044668	-587487014.3914438	-91855671.97164471	849139.5028543855	295817.07501475874	62472.274187524185	-3072626.3861645493	2542556.6771948826
18168276.39835402	1046135.4053310191	64371510.07652238	13857853.288172426	1896947.2930446756	1833783.048967673	-91855671.97164473	-17982927.874865077	180258.58592617506	46624.41439750799	63031.69997074727	-703771.1324359342	642805.4906908451
-86354321.53384532	-1

In [ ]:
dsignal_dpars_joint_ = np.concatenate([dsignal_dgbias50, dsignal_dgbias30, dsignal_dcosmic_multi])

np.shape(dsignal_dpars_joint_)

(18, 19, 612)

In [ ]:
FisherMatrixSKA_joint_ = fm.fisher_matrix(d_signal=dsignal_dpars_joint_, inv_cov=Inv_CovMatrixSKA_joint, z_bins=z_bins, bins=False)


 ... Computing the Fisher Matrix ... 

Signal & Cov must be given as functions (z,d) and (z,d,d') respectively. 


 ... Success! ... 



In [ ]:
np.shape(FisherMatrixSKA_joint_), np.linalg.det(FisherMatrixSKA_joint_)

((18, 18), 1.2322517742426492e+120)

In [ ]:
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in FisherMatrixSKA_joint_]))

239836850.62495834	48697266.23710122	170533246.63374412	18168276.35035656	-86354322.11230232	-19768479.423442256	-358913738.8152637	-53284837.76877848	354064026.31351715	176246927.39882022	-136201093.8359703	-1077569687.179304	208494937.07153094	-351869217.15498	-175149390.8182199	135330679.40578094	1070844588.7788311	-211864535.71184742
48697266.23710123	14351933.217645446	18168276.350356564	1046135.4021589902	-19768479.423442263	-6699195.871897356	-53284837.76877847	-10075359.004489357	59329304.975869335	29449576.562688127	-22535843.56127617	-180599721.44593447	28049502.7610185	-58577201.7918539	-29085868.673277378	22275980.644749835	178264993.86011094	-28708906.15959587
170533247.26709872	18168276.39835402	393454505.2658195	64371510.07652239	-22195058.948971387	1896947.293044668	-587487014.3914438	-91855671.97164471	426437443.39795786	214711291.63732675	-169688034.16453156	-1279274126.63607	285576678.0363355	-425588303.89510363	-214415474.56231198	169750506.43871903	1276201500.24990